In this notebook I will create a training table equivalent to ml_model_run_details

# Define Library

In [2]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.
# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
from google.cloud import storage
import os
import tempfile
import time
from datetime import datetime
import uuid
import joblib
import uuid

import gcsfs
import duckdb as dd
import pickle
import joblib
from typing import Union
import io
path = r'C:\Users\Dwaipayan\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')
os.environ["GOOGLE_CLOUD_PROJECT"] = "prj-prod-dataplatform"

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
pd.set_option('display.max_columns', None)
pd.set_option("Display.max_rows", 100)

### Function

#### expand_calc_features

In [3]:
import pandas as pd
import json

def expand_calc_features(df):
    """
    Expand the calcFeatures JSON column into separate columns and return the complete DataFrame.

    Parameters:
    df (pd.DataFrame): Input DataFrame with calcFeatures column containing JSON data

    Returns:
    pd.DataFrame: Expanded DataFrame with all original columns plus JSON features as separate columns
    """

    # Make a copy to avoid modifying the original DataFrame
    df_expanded = df.copy()

    # Parse the calcFeatures JSON column
    calc_features_list = []

    for idx, calc_features_str in enumerate(df['calcFeatures']):
        try:
            # Parse the JSON string
            features_dict = json.loads(calc_features_str.replace("'", '"'))  # Replace single quotes with double quotes for valid JSON
            calc_features_list.append(features_dict)
        except (json.JSONDecodeError, AttributeError) as e:
            # If parsing fails, create an empty dict and print warning
            print(f"Warning: Could not parse calcFeatures at index {idx}: {e}")
            calc_features_list.append({})

    # Create DataFrame from the parsed JSON data
    calc_features_df = pd.DataFrame(calc_features_list)

    # Add prefix to JSON-derived columns to avoid conflicts
    calc_features_df = calc_features_df.add_prefix('calc_')

    # Reset index to ensure proper alignment
    df_expanded = df_expanded.reset_index(drop=True)
    calc_features_df = calc_features_df.reset_index(drop=True)

    # Combine original DataFrame with expanded calcFeatures
    result_df = pd.concat([df_expanded, calc_features_df], axis=1)

    return result_df


#### expand_calc_features_robust

In [4]:
import pandas as pd
import json

def expand_calc_features_robust(df):
    """
    Expand the calcFeatures JSON column into separate columns with better error handling.

    Parameters:
    df (pd.DataFrame): Input DataFrame with calcFeatures column containing JSON data

    Returns:
    pd.DataFrame: Expanded DataFrame with all original columns plus JSON features as separate columns
    """

    # Make a copy to avoid modifying the original DataFrame
    df_expanded = df.copy()

    # Parse the calcFeatures JSON column
    calc_features_data = []

    for idx, row in df.iterrows():
        calc_features_str = row['calcFeatures']

        if pd.isna(calc_features_str) or calc_features_str == '':
            calc_features_data.append({})
            continue

        try:
            # Clean the string and parse JSON
            cleaned_str = calc_features_str.replace("'", '"').replace('None', 'null').replace('True', 'true').replace('False', 'false')
            features_dict = json.loads(cleaned_str)
            calc_features_data.append(features_dict)
        except Exception as e:
            print(f"Warning: Could not parse calcFeatures at index {idx}: {e}")
            print(f"Problematic string: {calc_features_str[:100]}...")  # Print first 100 chars
            calc_features_data.append({})

    # Create DataFrame from the parsed JSON data
    calc_features_df = pd.DataFrame(calc_features_data)

    # Add prefix to JSON-derived columns to avoid conflicts with existing columns
    calc_features_df = calc_features_df.add_prefix('feat_')

    # Combine DataFrames
    result_df = pd.concat([df_expanded, calc_features_df], axis=1)

    print(f"Original DataFrame shape: {df.shape}")
    print(f"Expanded DataFrame shape: {result_df.shape}")
    print(f"Added {len(calc_features_df.columns)} new columns from calcFeatures")

    return result_df

#### transform_data

In [5]:
# import pandas as pd
# import json
# import uuid
# from datetime import datetime
# from typing import List

# def transform_data(d1: pd.DataFrame, feature_column: List[str], a='demo_score', modelDisplayName = 'Cash_beta_trench1_Demo_backscore', subscription_name = 'sil_march 25 models'):
#     # Read the input CSV file
#     df = d1.copy()
    
#     # Create the output DataFrame with the required structure
#     output_data = []
    
#     for _, row in df.iterrows():
#         # Create the calcFeature JSON with all the feature columns
#         feature_columns = feature_column
        
#         calc_feature = {}
#         for col in feature_columns:
#             if col in row and pd.notna(row[col]):
#                 # Convert Timestamp objects to string
#                 if isinstance(row[col], pd.Timestamp):
#                     calc_feature[col] = row[col].isoformat()
#                 else:
#                     calc_feature[col] = row[col]
        
       
#         # Get current timestamp
#         current_time = datetime.now().isoformat()
        
#         # Create the output row
#         output_row = {
#             "customerId": row['customer_id'],
#             "digitalLoanAccountId": row['digitalLoanAccountId'],
#             "crifApplicationId": str(uuid.uuid4()),  # Generate random UUID
#             "prediction": row.get(a, 0),
#             "start_time": current_time,
#             "end_time": current_time,
#             "modelDisplayName":modelDisplayName,
#             "modelVersionId":"v1",
#             "subscription_name": subscription_name,
#             "message_id": str(uuid.uuid4()),  # Generate random UUID
#             "publish_time": current_time,
#             "attributes": "{}",  # Empty JSON object
#             "calcFeature": json.dumps(calc_feature, default=str)  # Use default=str to handle non-serializable objects
            
#         }
        
#         output_data.append(output_row)
    
#     # Create DataFrame from the output data
#     output_df = pd.DataFrame(output_data)
    
#     return output_df

# # Example usage:
# # transformeddata = 'cash_beta_trench1_applied_loans_backscored_20241001_20250831'
# # transform_data(f'{LOCALPATH}/{transformeddata}.csv')

In [6]:
import pandas as pd
import json
import uuid
from datetime import datetime
from typing import List

def transform_data(
    d1: pd.DataFrame, 
    feature_column: List[str], 
    a: str = 'demo_score', 
    modelDisplayName: str = 'Cash_beta_trench1_Demo_backscore', 
    tc: str = "", 
    subscription_name: str = 'sil_march 25 models'
) -> pd.DataFrame:
    """
    Transforms input data into a structured format suitable for model scoring output.

    Parameters:
    - d1 (pd.DataFrame): Input DataFrame containing raw data.
    - feature_column (List[str]): List of column names to include in the 'calcFeature' JSON.
    - a (str): Column name containing the prediction score. Default is 'demo_score'.
    - modelDisplayName (str): Name of the model used for scoring.
    - tc (str): Trench category (optional).
    - do (str): Device operating system. Default is 'android'.
    - subscription_name (str): Name of the subscription or model group.

    Returns:
    - pd.DataFrame: Transformed DataFrame with structured output.
    """

    # Make a copy of the input DataFrame to avoid modifying the original
    df = d1.copy()
    
    # Initialize an empty list to store transformed rows
    output_data = []
    
    # Iterate over each row in the DataFrame
    for _, row in df.iterrows():
        # Initialize dictionary to hold feature values
        calc_feature = {}
        
        # Loop through each feature column and extract its value from the row
        for col in feature_column:
            if col in row and pd.notna(row[col]):
                # Convert datetime values to ISO format strings
                if isinstance(row[col], pd.Timestamp):
                    calc_feature[col] = row[col].isoformat()
                else:
                    calc_feature[col] = row[col]
        
        # Get the current timestamp for start_time, end_time, and publish_time
        current_time = datetime.now().isoformat()
        
        # Construct the output row dictionary with required fields
        output_row = {
            "customerId": row['customer_id'],  # Unique customer identifier
            "digitalLoanAccountId": row['digitalLoanAccountId'],  # Loan account ID
            "crifApplicationId": str(uuid.uuid4()),  # Random UUID for application ID
            "prediction": row.get(a, 0),  # Prediction score from specified column
            "start_time": current_time,  # Timestamp when processing starts
            "end_time": current_time,    # Timestamp when processing ends
            "modelDisplayName": modelDisplayName,  # Name of the model used
            "modelVersionId": "v1",  # Static model version
            "calcFeature": json.dumps(calc_feature, default=str),  # Features as JSON string
            "subscription_name": subscription_name,  # Subscription name
            "message_id": str(uuid.uuid4()),  # Random UUID for message ID
            "publish_time": current_time,  # Timestamp when message is published
            "attributes": "{}",  # Placeholder for additional attributes
            "trenchCategory": tc,  # Optional trench category
            "deviceOs": row['osType'],  # Device operating system
        }
        
        # Append the transformed row to the output list
        output_data.append(output_row)
    
    # Convert the list of dictionaries to a DataFrame
    output_df = pd.DataFrame(output_data)
    
    # Return the transformed DataFrame
    return output_df


#### PSI Functions new

In [7]:
## Updated on 27-10-2025 - Modified for Training Period Baseline
import pandas as pd
import numpy as np
from typing import List, Dict, Tuple
import warnings
warnings.filterwarnings('ignore')

def identify_feature_types(df: pd.DataFrame, feature_list: List[str]) -> Dict[str, List[str]]:
    """
    Identify categorical and numerical features from the feature list.

    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe
    feature_list : List[str]
        List of features to classify

    Returns:
    --------
    Dict with 'categorical' and 'numerical' keys containing respective feature lists
    """
    categorical_features = []
    numerical_features = []

    for feature in feature_list:
        if feature not in df.columns:
            print(f"Warning: Feature '{feature}' not found in dataframe")
            continue

        # Check if feature is numeric
        if pd.api.types.is_numeric_dtype(df[feature]):
            # If unique values are less than 15 and all integers, treat as categorical
            unique_vals = df[feature].nunique()
            if unique_vals < 15 and df[feature].dropna().apply(lambda x: x == int(x) if isinstance(x, (int, float)) else False).all():
                categorical_features.append(feature)
            else:
                numerical_features.append(feature)
        else:
            categorical_features.append(feature)

    return {
        'categorical': categorical_features,
        'numerical': numerical_features
    }


def create_bins_for_features(df: pd.DataFrame,
                             numerical_features: List[str],
                             categorical_features: List[str],
                             train_period_df: pd.DataFrame) -> Dict:
    """
    Create bins for numerical features (deciles with fallback) and categorical features (top 6 + others)
    based on the entire training period data.

    Parameters:
    -----------
    df : pd.DataFrame
        Full input dataframe
    numerical_features : List[str]
        List of numerical features
    categorical_features : List[str]
        List of categorical features
    train_period_df : pd.DataFrame
        Training period dataframe (June 2024 to March 2025)

    Returns:
    --------
    Dictionary containing binning information for each feature
    """
    binning_info = {}

    # Create bins for numerical features with fallback strategy
    for feature in numerical_features:
        valid_data = train_period_df[feature].dropna()

        if len(valid_data) == 0:
            binning_info[feature] = {'type': 'numerical', 'bins': None, 'bin_ranges': {}}
            continue

        bins = None
        bin_count = None

        # Try 10 bins (deciles)
        try:
            test_bins = np.percentile(valid_data, np.arange(0, 101, 10))
            test_bins = np.unique(test_bins)
            if len(test_bins) >= 11:  # 11 edges = 10 bins
                bins = test_bins
                bin_count = 10
        except Exception as e:
            pass

        # If 10 bins not possible, try 5 bins
        if bins is None:
            try:
                test_bins = np.percentile(valid_data, np.arange(0, 101, 20))
                test_bins = np.unique(test_bins)
                if len(test_bins) >= 6:  # 6 edges = 5 bins
                    bins = test_bins
                    bin_count = 5
            except Exception as e:
                pass

        # If 5 bins not possible, try 3 bins
        if bins is None:
            try:
                test_bins = np.percentile(valid_data, [0, 33.33, 66.67, 100])
                test_bins = np.unique(test_bins)
                if len(test_bins) >= 4:  # 4 edges = 3 bins
                    bins = test_bins
                    bin_count = 3
            except Exception as e:
                pass

        # If still no bins possible, use equal distance bins of 5
        if bins is None:
            print(f"Warning: Feature '{feature}' has insufficient variance - cannot create standard bins")
            print(f"Feature '{feature}': Using equal distance bins of 5")

            min_val = valid_data.min()
            max_val = valid_data.max()

            # Create 5 equal distance bins
            bins = np.linspace(min_val, max_val, 6)  # 6 edges = 5 bins
            bins = np.unique(bins)
            bin_count = len(bins) - 1

            # If all values are the same, add slight buffer
            if bin_count == 1:
                bins = np.array([min_val - 0.1, min_val, min_val + 0.1])
                bin_count = 2
                print(f"Feature '{feature}': Constant value ({min_val}). Created 2 equal distance bins with buffer")

        # Add infinity edges to capture all values
        bins = bins.copy()
        bins[0] = -np.inf
        bins[-1] = np.inf

        print(f"Feature '{feature}': Created {bin_count} bins")

        # Create bin ranges dictionary
        bin_ranges = {}
        for i in range(len(bins)-1):
            bin_name = f"Bin_{i+1}"
            bin_ranges[bin_name] = {
                'min': bins[i],
                'max': bins[i+1],
                'range_str': f"[{bins[i]:.2f}, {bins[i+1]:.2f}]" if not np.isinf(bins[i]) and not np.isinf(bins[i+1]) else f"({bins[i]}, {bins[i+1]})"
            }

        binning_info[feature] = {
            'type': 'numerical',
            'bins': bins,
            'bin_ranges': bin_ranges,
            'bin_count': bin_count
        }

    # Create bins for categorical features (top 6 + others) using training period
    for feature in categorical_features:
        value_counts = train_period_df[feature].value_counts()
        unique_categories = value_counts.index.tolist()
        print(f"Unique categories: {unique_categories}")

        if len(unique_categories) <= 6:
            # Treat each category as a separate bin
            top_categories = unique_categories
        else:
            # Use top 6 categories only
            top_categories = value_counts.nlargest(6).index.tolist()

        print(f"Top categories for feature '{feature}': {top_categories}")

        binning_info[feature] = {
                'type': 'categorical',
                'top_categories': top_categories,
                'bin_ranges': {}  # No ranges for categorical
            }

    return binning_info


def apply_binning(df: pd.DataFrame,
                  feature: str,
                  binning_info: Dict) -> pd.Series:
    """
    Apply binning to a feature based on binning information.

    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe
    feature : str
        Feature name
    binning_info : Dict
        Binning information for the feature

    Returns:
    --------
    pd.Series with binned values
    """
    if binning_info['type'] == 'numerical':
        if binning_info['bins'] is None:
            return pd.Series(['Missing'] * len(df), index=df.index)

        bins = binning_info['bins']
        labels = [f"Bin_{i+1}" for i in range(len(bins)-1)]

        binned = pd.cut(df[feature],
                       bins=bins,
                       labels=labels,
                       include_lowest=True,
                       duplicates='drop')

        # Handle nulls - convert to string and then replace
        binned = binned.astype(str)
        binned[df[feature].isna()] = 'Missing'

        return binned

    else:  # categorical
        top_cats = binning_info['top_categories']

        # Convert to string for consistent comparison
        if pd.api.types.is_categorical_dtype(df[feature]):
            feature_data = df[feature].astype(str)
        else:
            feature_data = df[feature].astype(str)

        # Replace NaN string representation with 'Missing'
        feature_data = feature_data.replace('nan', 'Missing')

        # Convert top_cats to strings for comparison
        top_cats_str = [str(cat) for cat in top_cats]

        # Apply binning logic: use category name if in top_cats, else 'Others' (except for Missing)
        binned = feature_data.apply(lambda x: x if x in top_cats_str else ('Others' if x != 'Missing' else 'Missing'))

        return binned


def calculate_psi(expected_pct: pd.Series,
                  actual_pct: pd.Series,
                  epsilon: float = 0.0001) -> float:
    """
    Calculate Population Stability Index with proper epsilon handling and renormalization.

    Parameters:
    -----------
    expected_pct : pd.Series
        Expected (baseline) percentages
    actual_pct : pd.Series
        Actual percentages
    epsilon : float
        Small value to avoid log(0)

    Returns:
    --------
    PSI value
    """
    # Align indices
    all_bins = expected_pct.index.union(actual_pct.index)
    expected_pct = expected_pct.reindex(all_bins, fill_value=0)
    actual_pct = actual_pct.reindex(all_bins, fill_value=0)

    # Only add epsilon where values are zero
    expected_pct = expected_pct.apply(lambda x: epsilon if x == 0 else x)
    actual_pct = actual_pct.apply(lambda x: epsilon if x == 0 else x)

    # Renormalize to ensure they sum to 1 after adding epsilon
    expected_pct = expected_pct / expected_pct.sum()
    actual_pct = actual_pct / actual_pct.sum()

    # Calculate PSI
    psi_value = np.sum((actual_pct - expected_pct) * np.log(actual_pct / expected_pct))

    return psi_value


def calculate_month_on_month_psi(df: pd.DataFrame,
                                 feature_list: List[str],
                                 segment_columns: List[str],
                                 month_col: str = 'Application_month',
                                 data_selection_col: str = 'Data_selection',
                                 account_id_col: str = 'digitalLoanAccountId') -> pd.DataFrame:
    """
    Calculate PSI for each feature comparing training period (June 2024 to March 2025)
    vs each month after March 2025, overall and by segments.

    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe
    feature_list : List[str]
        List of features to calculate PSI for
    segment_columns : List[str]
        List of segment columns
    month_col : str
        Name of month column
    data_selection_col : str
        Name of data selection column (identifies train period)
    account_id_col : str
        Name of account ID column for counting distinct accounts

    Returns:
    --------
    pd.DataFrame with PSI values with one row per feature-month-segment combination
    """
    # Create a copy to avoid modifying original
    df = df.copy()

    # Identify training and test periods
    train_df = df[df[data_selection_col] == 'Train'].copy()
    test_df = df[df[data_selection_col] != 'Train'].copy()

    if len(train_df) == 0:
        raise ValueError("No training data found. Check Data_selection column.")

    print(f"Training period: {train_df[month_col].min()} to {train_df[month_col].max()}")
    print(f"Test period: {test_df[month_col].min()} to {test_df[month_col].max()}")

    # Identify feature types
    feature_types = identify_feature_types(df, feature_list)

    # Create binning strategy based on training period
    binning_info = create_bins_for_features(
        df,
        feature_types['numerical'],
        feature_types['categorical'],
        train_df
    )

    # Get sorted test months
    test_months = sorted(test_df[month_col].unique())

    results = []

    # Calculate overall PSI
    for feature in feature_list:
        if feature not in df.columns:
            continue

        # Apply binning to entire dataset
        df[f'{feature}_binned'] = apply_binning(df, feature, binning_info[feature])
        # print(f"Feature binned {df[f'{feature}_binned']}")
        # Get training period distribution (baseline)
        train_baseline = df[df[data_selection_col] == 'Train'][f'{feature}_binned'].value_counts(normalize=True)

        # Calculate PSI for each test month
        for month in test_months:
            actual_dist = df[df[month_col] == month][f'{feature}_binned'].value_counts(normalize=True)
            psi_value = calculate_psi(train_baseline, actual_dist)

            # Calculate average percentages across all bins
            expected_avg_pct = train_baseline.mean() * 100
            actual_avg_pct = actual_dist.mean() * 100

            # # Count distinct accounts for segment
            # base_segment_count = train_segment[account_id_col].nunique()
            # actual_segment_count = actual_segment[account_id_col].nunique()


            results.append({
                'Feature': feature,
                'Feature_Type': binning_info[feature]['type'],
                'Segment_Column': 'Overall',
                'Segment_Value': 'All',
                'Month': f"{month}",
                'Base_Month': 'Train (Jun 2024 - Mar 2025)',
                'Current_Month': month,
                'Expected_Percentage': expected_avg_pct,
                'Actual_Percentage': actual_avg_pct,
                'PSI': psi_value
            })

    # Calculate PSI by segments
    for segment_col in segment_columns:
        if segment_col not in df.columns:
            continue

        segments = df[segment_col].dropna().unique()

        for segment_val in segments:
            segment_df = df[df[segment_col] == segment_val]

            for feature in feature_list:
                if feature not in df.columns:
                    continue

                # Get training period distribution for segment
                train_segment = segment_df[segment_df[data_selection_col] == 'Train']
                if len(train_segment) == 0:
                    continue

                train_baseline = train_segment[f'{feature}_binned'].value_counts(normalize=True)

                # Calculate PSI for each test month
                for month in test_months:
                    actual_segment = segment_df[segment_df[month_col] == month]
                    if len(actual_segment) == 0:
                        continue

                    actual_dist = actual_segment[f'{feature}_binned'].value_counts(normalize=True)
                    psi_value = calculate_psi(train_baseline, actual_dist)

                    # Calculate average percentages across all bins
                    expected_avg_pct = train_baseline.mean() * 100
                    actual_avg_pct = actual_dist.mean() * 100

                    # Count distinct accounts for segment
                    base_segment_count = train_segment[account_id_col].nunique()
                    actual_segment_count = actual_segment[account_id_col].nunique()

                    results.append({
                        'Feature': feature,
                        'Feature_Type': binning_info[feature]['type'],
                        'Segment_Column': segment_col,
                        'Segment_Value': segment_val,
                        'Month': f"{month}",
                        'Base_Month': 'Train (Jun 2024 - Mar 2025)',
                        'Current_Month': month,
                        'Base_Count': base_segment_count,
                        'Actual_Count': actual_segment_count,
                        'Expected_Percentage': expected_avg_pct,
                        'Actual_Percentage': actual_avg_pct,
                        'PSI': psi_value
                    })

    return pd.DataFrame(results)


def calculate_bin_level_psi(df: pd.DataFrame,
                            feature_list: List[str],
                            segment_columns: List[str],
                            month_col: str = 'Application_month',
                            data_selection_col: str = 'Data_selection',
                            account_id_col: str = 'digitalLoanAccountId') -> pd.DataFrame:
    """
    Calculate bin-level PSI for each feature comparing training period
    vs each month after March 2025, overall and by segments.

    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe
    feature_list : List[str]
        List of features to calculate PSI for
    segment_columns : List[str]
        List of segment columns
    month_col : str
        Name of month column
    data_selection_col : str
        Name of data selection column
    account_id_col : str
        Name of account ID column for counting distinct accounts

    Returns:
    --------
    pd.DataFrame with bin-level PSI details including bin ranges
    """
    # Create a copy to avoid modifying original
    df = df.copy()

    # Identify training and test periods
    train_df = df[df[data_selection_col] == 'Train'].copy()
    test_df = df[df[data_selection_col] != 'Train'].copy()

    if len(train_df) == 0:
        raise ValueError("No training data found. Check Data_selection column.")

    print(f"Training period: {train_df[month_col].min()} to {train_df[month_col].max()}")
    print(f"Test period: {test_df[month_col].min()} to {test_df[month_col].max()}")

    # Identify feature types
    feature_types = identify_feature_types(df, feature_list)

    # Create binning strategy based on training period
    binning_info = create_bins_for_features(
        df,
        feature_types['numerical'],
        feature_types['categorical'],
        train_df
    )

    # Get sorted test months
    test_months = sorted(test_df[month_col].unique())

    results = []
    epsilon = 0.0001

    # Calculate overall bin-level PSI
    for feature in feature_list:
        if feature not in df.columns:
            continue

        # Apply binning to entire dataset
        df[f'{feature}_binned'] = apply_binning(df, feature, binning_info[feature])
        # print(df[f'{feature}_binned'])

        # Get training period distribution (baseline)
        train_baseline = df[df[data_selection_col] == 'Train'][f'{feature}_binned'].value_counts(normalize=True)

        # Calculate bin-level PSI for each test month
        for month in test_months:
            month_data = df[df[month_col] == month]
            actual_dist = month_data[f'{feature}_binned'].value_counts(normalize=True)

            # Count distinct accounts
            base_count = df[df[data_selection_col] == 'Train'][account_id_col].nunique()
            actual_count = month_data[account_id_col].nunique()

            # Get all bins
            all_bins = train_baseline.index.union(actual_dist.index)

            for bin_name in all_bins:
                # Simplified epsilon logic - no redundancy
                expected_pct = train_baseline.get(bin_name, 0)
                actual_pct = actual_dist.get(bin_name, 0)

                # Add epsilon only if zero
                expected_pct = epsilon if expected_pct == 0 else expected_pct
                actual_pct = epsilon if actual_pct == 0 else actual_pct

                # Calculate bin-level PSI
                bin_psi = (actual_pct - expected_pct) * np.log(actual_pct / expected_pct)

                # Get bin range information
                bin_ranges = binning_info[feature]['bin_ranges']
                if bin_name in bin_ranges:
                    bin_min = bin_ranges[bin_name]['min']
                    bin_max = bin_ranges[bin_name]['max']
                    bin_range = bin_ranges[bin_name]['range_str']
                else:
                    # For categorical or special bins (Missing, Others)
                    bin_min = None
                    bin_max = None
                    bin_range = bin_name

                results.append({
                    'Feature': feature,
                    'Feature_Type': binning_info[feature]['type'],
                    'Segment_Column': 'Overall',
                    'Segment_Value': 'All',
                    'Month': f"{month}",
                    'Base_Month': 'Train (Jun 2024 - Mar 2025)',
                    'Current_Month': month,
                    'Base_Count': base_count,
                    'Actual_Count': actual_count,
                    'Bin': bin_name,
                    'Bin_Range': bin_range,
                    'Bin_Min': bin_min,
                    'Bin_Max': bin_max,
                    'Base_Percentage': (train_baseline.get(bin_name, 0) * 100),
                    'Actual_Percentage': (actual_dist.get(bin_name, 0) * 100),
                    'Bin_PSI': bin_psi
                })

    # Calculate bin-level PSI by segments
    for segment_col in segment_columns:
        if segment_col not in df.columns:
            continue

        segments = df[segment_col].dropna().unique()

        for segment_val in segments:
            segment_df = df[df[segment_col] == segment_val]

            for feature in feature_list:
                if feature not in df.columns:
                    continue

                # Get training period distribution for segment
                train_segment = segment_df[segment_df[data_selection_col] == 'Train']
                if len(train_segment) == 0:
                    continue

                train_baseline = train_segment[f'{feature}_binned'].value_counts(normalize=True)

                # Calculate bin-level PSI for each test month
                for month in test_months:
                    actual_segment = segment_df[segment_df[month_col] == month]
                    if len(actual_segment) == 0:
                        continue

                    actual_dist = actual_segment[f'{feature}_binned'].value_counts(normalize=True)

                    # Count distinct accounts for segment
                    base_segment_count = train_segment[account_id_col].nunique()
                    actual_segment_count = actual_segment[account_id_col].nunique()

                    # Get all bins
                    all_bins = train_baseline.index.union(actual_dist.index)

                    for bin_name in all_bins:
                        # Simplified epsilon logic - no redundancy
                        expected_pct = train_baseline.get(bin_name, 0)
                        actual_pct = actual_dist.get(bin_name, 0)

                        # Add epsilon only if zero
                        expected_pct = epsilon if expected_pct == 0 else expected_pct
                        actual_pct = epsilon if actual_pct == 0 else actual_pct

                        # Calculate bin-level PSI
                        bin_psi = (actual_pct - expected_pct) * np.log(actual_pct / expected_pct)

                        # Get bin range information
                        bin_ranges = binning_info[feature]['bin_ranges']
                        if bin_name in bin_ranges:
                            bin_min = bin_ranges[bin_name]['min']
                            bin_max = bin_ranges[bin_name]['max']
                            bin_range = bin_ranges[bin_name]['range_str']
                        else:
                            # For categorical or special bins (Missing, Others)
                            bin_min = None
                            bin_max = None
                            bin_range = bin_name

                        results.append({
                            'Feature': feature,
                            'Feature_Type': binning_info[feature]['type'],
                            'Segment_Column': segment_col,
                            'Segment_Value': segment_val,
                            'Month': f"{month}",
                            'Base_Month': 'Train (Jun 2024 - Mar 2025)',
                            'Current_Month': month,
                            'Base_Count': base_segment_count,
                            'Actual_Count': actual_segment_count,
                            'Bin': bin_name,
                            'Bin_Range': bin_range,
                            'Bin_Min': bin_min,
                            'Bin_Max': bin_max,
                            'Base_Percentage': (train_baseline.get(bin_name, 0) * 100),
                            'Actual_Percentage': (actual_dist.get(bin_name, 0) * 100),
                            'Bin_PSI': bin_psi
                        })

    return pd.DataFrame(results)

## SIL

#### Query from risk_mart.sil_risk_ds_master_20230101_20250309_v2

##### 'Alpha - CIC-SIL-Model'

In [8]:
sq = """select distinct
    r.customerId customer_id ,
    r.digitalLoanAccountId,
    r.cic_score,
    r.cic_Personal_Loans_granted_contracts_amt_24M,
    r.cic_days_since_last_inquiry, 
    r.cic_cnt_active_contracts,
    r.cic_vel_contract_nongranted_cnt_12on24,
    r.cic_max_amt_granted_24M, 
    r.cic_zero_non_granted_ever_flag,
    r.cic_tot_active_contracts_util,
    r.cic_vel_contract_granted_amt_12on24,
    r.cic_zero_granted_ever_flag,
    case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
from risk_mart.sil_risk_ds_master_20230101_20250309_v2 r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId

where cic_score is not null;"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID e95b97d2-fab0-41c9-96e4-980f8b4fe9f4 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (204453, 13)


In [9]:
feature_column = ['cic_Personal_Loans_granted_contracts_amt_24M',
       'cic_days_since_last_inquiry', 'cic_cnt_active_contracts',
       'cic_vel_contract_nongranted_cnt_12on24', 'cic_max_amt_granted_24M',
       'cic_zero_non_granted_ever_flag', 'cic_tot_active_contracts_util',
       'cic_vel_contract_granted_amt_12on24', 'cic_zero_granted_ever_flag']

In [10]:
dfd = transform_data(data, feature_column, a='cic_score', modelDisplayName='Alpha - CIC-SIL-Model') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,2591981,0f71ab25-0be6-4237-8d8e-4f166ee49169,f8f3923d-f0c8-4779-a8e8-448404077fe3,0.126862,2025-11-28T13:42:43.351477,2025-11-28T13:42:43.351477,Alpha - CIC-SIL-Model,v1,"{""cic_days_since_last_inquiry"": 1825.0, ""cic_z...",sil_march 25 models,19ab0940-aadf-4992-a0a2-d51e28c8479b,2025-11-28T13:42:43.351477,{},,android
1,2482970,3f9365df-7375-4c14-a28b-86a26b423a84,df367e25-ec50-45dd-8add-2d6f82cb114d,0.098358,2025-11-28T13:42:43.351477,2025-11-28T13:42:43.351477,Alpha - CIC-SIL-Model,v1,"{""cic_days_since_last_inquiry"": 174.0, ""cic_cn...",sil_march 25 models,2fdaa201-a2c6-44bb-aae5-3984dd2332f0,2025-11-28T13:42:43.351477,{},,android
2,2817917,2196fe98-95a6-4bfb-856c-5342f6bccf8f,11f5246d-d353-459f-8840-b41de939c09f,0.124686,2025-11-28T13:42:43.351477,2025-11-28T13:42:43.351477,Alpha - CIC-SIL-Model,v1,"{""cic_days_since_last_inquiry"": 1183.0, ""cic_z...",sil_march 25 models,830e99fd-06e5-48f0-9460-61f028002deb,2025-11-28T13:42:43.351477,{},,android
3,3262358,a383b3dd-28e1-4607-848b-e1a57c07a91a,2640ce0d-a657-4f01-9ccd-c6d9348fec04,0.173223,2025-11-28T13:42:43.351477,2025-11-28T13:42:43.351477,Alpha - CIC-SIL-Model,v1,"{""cic_days_since_last_inquiry"": 67.0, ""cic_vel...",sil_march 25 models,4c3182a9-4e56-4baf-9bb5-e80046a8cc2f,2025-11-28T13:42:43.351477,{},,android
4,3063565,a9bc00d9-196f-4591-bab2-47dfbf1805e0,a13d235e-a001-45a4-bbcc-e921136bcf0f,0.085567,2025-11-28T13:42:43.352472,2025-11-28T13:42:43.352472,Alpha - CIC-SIL-Model,v1,"{""cic_Personal_Loans_granted_contracts_amt_24M...",sil_march 25 models,cf8dd11c-37c9-4bfb-a912-ac83515a2045,2025-11-28T13:42:43.352472,{},,android


In [11]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=768bb745-e1fa-4c2c-82d5-34f83a4f4f10>

##### Alpha Sil Stack Model 

In [12]:
sq = """ 
select distinct 
r.customerId customer_id ,
r.digitalLoanAccountId,
r.alpha_stack_score,
r.beta_demo_score,
r.cic_score,
r.apps_score,
r.credo_gen_score,
    case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
from `risk_mart.sil_risk_ds_master_20230101_20250309_v2` r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
where alpha_stack_score is not null
;
"""

data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 4dcabc22-395a-4e23-9a0c-29c7960da6c6 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (204453, 8)


In [13]:
feature_column = ['beta_demo_score',
       'cic_score', 'apps_score',
       'credo_gen_score']

dfd = transform_data(data, feature_column, a='alpha_stack_score', modelDisplayName='Alpha - StackingModel') 
dfd.head()


,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,2424505,dc4ed5d8-69fb-4fb8-a132-8e07ead7ae93,24b2184c-136b-48c0-8de8-bb1a7b5fd2a7,0.059519,2025-11-28T13:43:14.780526,2025-11-28T13:43:14.780526,Alpha - StackingModel,v1,"{""beta_demo_score"": 0.06515353039530854, ""cic_...",sil_march 25 models,a29cd994-60b6-44db-8797-8d7a20d883b8,2025-11-28T13:43:14.780526,{},,android
1,3023906,d0af11a8-2367-478b-9c19-69cd70d54942,2070efcb-1eb5-4ffa-8adc-40265c20273b,0.055909,2025-11-28T13:43:14.781526,2025-11-28T13:43:14.781526,Alpha - StackingModel,v1,"{""beta_demo_score"": 0.0713566097707466, ""cic_s...",sil_march 25 models,d5fd7711-f06e-4c04-878c-e412fb7ee036,2025-11-28T13:43:14.781526,{},,android
2,2665520,f662dc19-187f-47e7-b07e-c288714060d4,febf5d94-eef6-4797-8d38-99046fd8049e,0.085580,2025-11-28T13:43:14.781526,2025-11-28T13:43:14.781526,Alpha - StackingModel,v1,"{""beta_demo_score"": 0.09625384281300788, ""cic_...",sil_march 25 models,dd6c6d9e-723e-467a-8084-97561519031d,2025-11-28T13:43:14.781526,{},,android
3,2817917,2196fe98-95a6-4bfb-856c-5342f6bccf8f,23dcfacc-b9b5-4740-80a6-52e6161c39af,0.257212,2025-11-28T13:43:14.781526,2025-11-28T13:43:14.781526,Alpha - StackingModel,v1,"{""beta_demo_score"": 0.1350818237118101, ""cic_s...",sil_march 25 models,546bc050-528b-43c5-972b-43bfe17a2fd9,2025-11-28T13:43:14.781526,{},,android
4,2348760,b97e6d9a-09b5-41e8-adce-0d03665873d5,17c47099-6a9c-427d-8513-f80515fa6758,0.122655,2025-11-28T13:43:14.781526,2025-11-28T13:43:14.781526,Alpha - StackingModel,v1,"{""beta_demo_score"": 0.14179810818976543, ""cic_...",sil_march 25 models,ac371ccd-2758-492b-94e1-254d83faac38,2025-11-28T13:43:14.781526,{},,android


In [14]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=a0c1ef3a-56db-4c85-9ca3-0c47cf6d9ea3>

##### Beta Sil App Score

In [15]:
sq = """ 
select distinct
r.customerId customer_id ,
r.digitalLoanAccountId,
r.apps_score,
r.app_cnt_absence_tag_30d,
r.app_cnt_absence_tag_90d ,
r.app_cnt_business_ever ,
r.app_cnt_competitors_30d ,
r.app_cnt_competitors_90d ,
r.app_cnt_education_ever ,
r.app_cnt_finance_7d ,
r.app_cnt_finance_90d ,
r.app_cnt_music_and_audio_ever ,
r.app_cnt_payday_90d ,
r.app_cnt_rated_for_3plus_ever ,
r.app_cnt_travel_and_local_ever ,
r.app_first_competitors_install_to_apply_days ,
r.app_first_payday_install_to_apply_days ,
r.app_median_time_bw_installed_mins_30d ,
r.app_vel_finance_30_over_365 ,
    case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
from `risk_mart.sil_risk_ds_master_20230101_20250309_v2` r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
where apps_score is not null
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 1664026d-d33a-4a4f-8579-9dc290308951 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (317385, 20)


In [16]:
feature_column = ['app_cnt_rated_for_3plus_ever',
       'app_cnt_education_ever', 'app_cnt_business_ever',
       'app_cnt_music_and_audio_ever',
       'app_cnt_travel_and_local_ever', 'app_cnt_finance_7d',
       'app_cnt_competitors_30d', 'app_cnt_absence_tag_30d',
        'app_cnt_absence_tag_90d',
       'app_cnt_finance_90d', 'app_cnt_competitors_90d',
       'app_cnt_payday_90d',
       'app_median_time_bw_installed_mins_30d',
       'app_first_competitors_install_to_apply_days',
       'app_first_payday_install_to_apply_days',
       'app_vel_finance_30_over_365']

dfd = transform_data(data, feature_column, a='apps_score', modelDisplayName='Beta - AppsScoreModel') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,3265787,2fa5501c-fee8-49ce-b0fc-7598e7499d10,6ac08672-7e08-4d46-a53b-e8f665caf3a4,0.670536,2025-11-28T13:43:45.702007,2025-11-28T13:43:45.702007,Beta - AppsScoreModel,v1,"{""app_cnt_rated_for_3plus_ever"": 39.0, ""app_cn...",sil_march 25 models,3688962c-b5a1-4a89-9189-b976328d02a5,2025-11-28T13:43:45.702007,{},,android
1,3045258,a1f6d39b-915a-41f5-b5f8-969ae1d0412c,a69029d8-d42f-46e7-962c-a1ef7ff6be9d,0.533583,2025-11-28T13:43:45.702007,2025-11-28T13:43:45.702007,Beta - AppsScoreModel,v1,"{""app_cnt_rated_for_3plus_ever"": 38.0, ""app_cn...",sil_march 25 models,f6ce0581-dda0-440d-9af0-d986ce676bd4,2025-11-28T13:43:45.702007,{},,android
2,2272982,8d848bbe-ef4d-4aaf-bb07-cde9166afe01,685a5540-84b3-42c9-b988-b475e754ac13,0.717967,2025-11-28T13:43:45.703019,2025-11-28T13:43:45.703019,Beta - AppsScoreModel,v1,"{""app_cnt_rated_for_3plus_ever"": 29.0, ""app_cn...",sil_march 25 models,2423faaf-4e32-434a-9939-2b9df8cd72a5,2025-11-28T13:43:45.703019,{},,android
3,2069036,ee6276eb-66a3-47a3-a4e8-06a8425ce82a,2feed125-8cec-4059-adce-e1e82bca6714,0.675898,2025-11-28T13:43:45.703019,2025-11-28T13:43:45.703019,Beta - AppsScoreModel,v1,"{""app_cnt_rated_for_3plus_ever"": 54.0, ""app_cn...",sil_march 25 models,4b28e70e-254b-46b6-aeee-a996eaf8c934,2025-11-28T13:43:45.703019,{},,android
4,3123422,9d297e5e-014b-458d-ba05-593c1c7e62ac,afd476db-782e-4c52-8ff4-7dcbf9caaad0,0.341867,2025-11-28T13:43:45.703019,2025-11-28T13:43:45.703019,Beta - AppsScoreModel,v1,"{""app_cnt_rated_for_3plus_ever"": 62.0, ""app_cn...",sil_march 25 models,2ade5db1-a62a-4331-9293-82b43189b6e8,2025-11-28T13:43:45.703019,{},,android


In [ ]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

##### Beta SIL Demo Score

In [ ]:
sq = """
select distinct
r.customerId customer_id ,
r.digitalLoanAccountId,
r.beta_demo_score,
r.beta_de_ln_vas_opted_flag ,
r.beta_de_ln_doc_type_rolled ,
r.beta_de_ln_marital_status ,
r.beta_de_ln_age_bin ,
r.beta_de_ln_province_bin ,
r.beta_de_ln_ref2_type ,
r.beta_de_ln_education_level ,
r.beta_de_ln_ref1_type ,
r.beta_de_ln_industry_new_bin ,
r.beta_de_ln_appln_day_of_week ,
r.beta_de_onb_name_email_match_score ,
r.beta_de_ln_employment_type_new_bin ,
r.beta_de_ln_telconame ,
r.beta_de_time_bw_onb_loan_appln_mins ,
r.beta_de_ln_source_of_funds_new_bin ,
r.beta_de_ln_brand_bin ,
r.beta_de_ln_email_primary_domain ,
    case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
from `risk_mart.sil_risk_ds_master_20230101_20250309_v2` r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
where beta_demo_score is not null
;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID e0ac770c-ee44-4510-acc0-7600b633ea86 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (349207, 21)


In [ ]:
feature_column = ['beta_de_ln_vas_opted_flag',
       'beta_de_ln_doc_type_rolled', 'beta_de_ln_marital_status',
       'beta_de_ln_age_bin', 'beta_de_ln_province_bin',
       'beta_de_ln_ref2_type', 'beta_de_ln_education_level',
       'beta_de_ln_ref1_type', 'beta_de_ln_industry_new_bin',
       'beta_de_ln_appln_day_of_week',
       'beta_de_onb_name_email_match_score',
       'beta_de_ln_employment_type_new_bin', 'beta_de_ln_telconame',
       'beta_de_time_bw_onb_loan_appln_mins',
       'beta_de_ln_source_of_funds_new_bin', 'beta_de_ln_brand_bin',
       'beta_de_ln_email_primary_domain']

dfd = transform_data(data, feature_column, a='beta_demo_score', modelDisplayName='Beta - DemoScoreModel') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,2473458,d7339f26-c516-489f-a011-1943d7a16f44,862e7b08-3e22-461d-9a58-8f0d06f18d05,0.121699,2025-11-27T04:31:35.179687,2025-11-27T04:31:35.179687,Beta - DemoScoreModel,v1,"{""beta_de_ln_vas_opted_flag"": ""1"", ""beta_de_ln...",sil_march 25 models,c4501567-e731-4ece-948e-f9456ba4ca2c,2025-11-27T04:31:35.179687,{},,android
1,1931382,7e9b947d-4c93-4d0d-8c06-11d60027a0c2,2b8270ea-1bc5-4a25-9280-d6acb18389e1,0.175571,2025-11-27T04:31:35.180052,2025-11-27T04:31:35.180052,Beta - DemoScoreModel,v1,"{""beta_de_ln_vas_opted_flag"": ""0"", ""beta_de_ln...",sil_march 25 models,7d320e8d-d15d-4d45-a5cf-ef44063b65b2,2025-11-27T04:31:35.180052,{},,ios
2,2178156,3aa09643-de0b-428a-a9b1-e0e5a32ef37f,38289ebd-aa8c-4c24-8118-1357e50bf236,0.069422,2025-11-27T04:31:35.180320,2025-11-27T04:31:35.180320,Beta - DemoScoreModel,v1,"{""beta_de_ln_vas_opted_flag"": ""0"", ""beta_de_ln...",sil_march 25 models,369295b3-b378-4a09-8e3e-349ea4d91a3d,2025-11-27T04:31:35.180320,{},,android
3,3297746,db65795c-c0fe-4ae6-a857-aa5c9770da15,c094230d-4cbf-4956-92b6-20ce8e9c6f82,0.041516,2025-11-27T04:31:35.180590,2025-11-27T04:31:35.180590,Beta - DemoScoreModel,v1,"{""beta_de_ln_vas_opted_flag"": ""1"", ""beta_de_ln...",sil_march 25 models,600b3ce4-d0d6-4980-ad7d-761ae246bc0f,2025-11-27T04:31:35.180590,{},,android
4,1980584,be9bec7b-9db8-4119-ab90-4243e13aef9b,8f364388-c5a9-468e-acae-35d8300a6e73,0.060127,2025-11-27T04:31:35.180840,2025-11-27T04:31:35.180840,Beta - DemoScoreModel,v1,"{""beta_de_ln_vas_opted_flag"": ""0"", ""beta_de_ln...",sil_march 25 models,f8489e69-582b-4bc2-b144-6c6da831b15a,2025-11-27T04:31:35.180840,{},,ios


In [ ]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=ef7c65ab-a6e8-49bc-b249-7ee02bd200ad>

##### Beta SIL STACK Score Model

In [ ]:
sq = """ 
select  distinct
r.customerId customer_id ,
r.digitalLoanAccountId,
r.beta_stack_score,
r.apps_score,
r.credo_gen_score,
r.beta_demo_score,
    case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
from `risk_mart.sil_risk_ds_master_20230101_20250309_v2` r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
where beta_stack_score is not null
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 29f93b84-d22b-4f50-b966-74138c7ed74e successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (349207, 7)


In [ ]:
feature_column = ['apps_score', 'credo_gen_score', 'beta_demo_score']
dfd = transform_data(data, feature_column, a='beta_stack_score', modelDisplayName='Beta - StackScoreModel') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,2507425,096b1208-6917-46ce-912a-a25f2b9f891d,2f0475f6-1a0a-47fa-9406-11d4c18cf1f9,0.143520,2025-11-27T04:33:25.917605,2025-11-27T04:33:25.917605,Beta - StackScoreModel,v1,"{""apps_score"": 0.629784603344141, ""credo_gen_s...",sil_march 25 models,257f5af8-6de1-4391-92a0-389805fa5717,2025-11-27T04:33:25.917605,{},,android
1,3065852,e9921095-718f-40b9-8533-6ffe0f5584cf,88f5aa5b-1a14-4906-ac0a-bff81746900f,0.074762,2025-11-27T04:33:25.917851,2025-11-27T04:33:25.917851,Beta - StackScoreModel,v1,"{""apps_score"": 0.5161863693480311, ""credo_gen_...",sil_march 25 models,33aff2d6-1bb6-4760-8be3-5bb41e516ec0,2025-11-27T04:33:25.917851,{},,android
2,2597773,0391a56d-6066-41ec-8b00-157ab1b21081,19f2aa45-2a8b-46ac-a6d3-36b1e2a566ba,0.123071,2025-11-27T04:33:25.918012,2025-11-27T04:33:25.918012,Beta - StackScoreModel,v1,"{""apps_score"": 0.5855399084897734, ""credo_gen_...",sil_march 25 models,3e1b7aa6-f004-42a7-822b-7e97f5eb27d5,2025-11-27T04:33:25.918012,{},,android
3,2721343,fb3fc361-3716-4743-9f6e-e9822b4c738b,e6e61cd3-c4b0-4ffa-b2e2-a5a4b7b09966,0.036088,2025-11-27T04:33:25.918154,2025-11-27T04:33:25.918154,Beta - StackScoreModel,v1,"{""apps_score"": 0.3964347248172607, ""credo_gen_...",sil_march 25 models,fff04e21-db31-463e-81d5-69b301856a58,2025-11-27T04:33:25.918154,{},,android
4,2594353,1b577cc6-6633-4a2e-afd8-c4938d6219a8,197e7e2e-0990-4ef9-9d91-12b95354b452,0.064225,2025-11-27T04:33:25.918290,2025-11-27T04:33:25.918290,Beta - StackScoreModel,v1,"{""apps_score"": 0.48833455608988097, ""credo_gen...",sil_march 25 models,92182c54-01a4-4f58-80c6-b72764842b52,2025-11-27T04:33:25.918290,{},,android


In [ ]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=33429c25-551a-47c0-87e5-de05435c64d8>

##### Alpha  - IncomeEstimationModel

In [ ]:
sq = """  
Select 
distinct
r.customerId customer_id ,
r.digitalLoanAccountId,
r.alpha_estimated_income,
r.inc_alpha_cic_credit_avg_credit_limit,
r.inc_alpha_cic_max_active_contracts_amt,
r.inc_alpha_ln_company_name,
r.inc_alpha_ln_age,
r.inc_alpha_doc_type_rolled,
r.inc_alpha_ln_brand,
r.inc_alpha_ln_city,
r.inc_alpha_ln_cnt_dependents,
r.inc_alpha_ln_education_level,
r.inc_alpha_ln_employment_type_new,
r.inc_alpha_ln_gender,
r.inc_alpha_ln_industry_new,
r.inc_alpha_ln_loan_prod_type,
r.inc_alpha_ln_marital_status_new,
r.inc_alpha_ln_nature_of_work_new,
r.inc_alpha_ln_osversion_bin,
r.inc_alpha_ln_purpose,
r.inc_alpha_ln_source_of_funds_new,
r.inc_alpha_loan_monthly_income,
r.inc_alpha_encoded_company_name_grouped,
    case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
from `risk_mart.sil_risk_ds_master_20230101_20250309_v2` r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
where r.alpha_estimated_income is not null
;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")


Job ID e0fb6535-33fd-4c0d-887d-233e617dfc41 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (349207, 24)


In [ ]:
feature_column = ['inc_alpha_cic_credit_avg_credit_limit',
       'inc_alpha_cic_max_active_contracts_amt', 'inc_alpha_ln_age',
       'inc_alpha_doc_type_rolled', 'inc_alpha_ln_brand', 'inc_alpha_ln_city',
       'inc_alpha_ln_cnt_dependents', 'inc_alpha_ln_education_level',
       'inc_alpha_ln_employment_type_new', 'inc_alpha_ln_gender',
       'inc_alpha_ln_industry_new', 'inc_alpha_ln_loan_prod_type',
       'inc_alpha_ln_marital_status_new', 'inc_alpha_ln_nature_of_work_new',
       'inc_alpha_ln_osversion_bin', 'inc_alpha_ln_purpose',
       'inc_alpha_ln_source_of_funds_new',
       'inc_alpha_encoded_company_name_grouped']
dfd = transform_data(data, feature_column, a='alpha_estimated_income', modelDisplayName='Alpha  - IncomeEstimationModel') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,2800859,ddc481ca-e674-4ac0-a55b-ffc98fa1fd5e,b249cf9d-5731-488e-bc56-6498a7e20458,8281.979169,2025-11-27T04:34:25.232979,2025-11-27T04:34:25.232979,Alpha - IncomeEstimationModel,v1,"{""inc_alpha_cic_credit_avg_credit_limit"": 0.0,...",sil_march 25 models,1013c4c1-7c89-4a71-9c99-c1bb9c2f1b0f,2025-11-27T04:34:25.232979,{},,android
1,2073530,cbc00c30-9f2c-42f0-a0ae-ac1c6b068354,161ad884-b499-4e5f-83ac-4f47c8c3e04b,9343.501809,2025-11-27T04:34:25.233381,2025-11-27T04:34:25.233381,Alpha - IncomeEstimationModel,v1,"{""inc_alpha_cic_credit_avg_credit_limit"": 0.0,...",sil_march 25 models,e04322b2-7539-4214-8c37-fe90f44ee6f3,2025-11-27T04:34:25.233381,{},,android
2,2437304,6e86c8bc-5264-4de8-82e9-abc3e3c24576,93606da6-f642-4470-bf45-7fd06da04e92,8657.650838,2025-11-27T04:34:25.233690,2025-11-27T04:34:25.233690,Alpha - IncomeEstimationModel,v1,"{""inc_alpha_cic_credit_avg_credit_limit"": 0.0,...",sil_march 25 models,2f90b4e1-6406-4b01-8e5d-5ec7abd087dd,2025-11-27T04:34:25.233690,{},,android
3,3107384,ed2c9dc4-ccca-407e-9133-f5ec71fbdad1,9c656175-7e46-4ad3-bc28-c6ee1e680532,8302.131841,2025-11-27T04:34:25.234021,2025-11-27T04:34:25.234021,Alpha - IncomeEstimationModel,v1,"{""inc_alpha_cic_credit_avg_credit_limit"": 0.0,...",sil_march 25 models,6cfece59-c14e-4495-92b5-63d45ca9d038,2025-11-27T04:34:25.234021,{},,android
4,2987653,fcc029b7-4c99-429f-a086-355ca1eb3db9,7eb67b33-733b-408d-a114-9f7e6709aea3,24146.554891,2025-11-27T04:34:25.234301,2025-11-27T04:34:25.234301,Alpha - IncomeEstimationModel,v1,"{""inc_alpha_cic_credit_avg_credit_limit"": 0.0,...",sil_march 25 models,27afcb4c-72d6-47fa-8054-c190779dd2df,2025-11-27T04:34:25.234301,{},,android


In [ ]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=04b81120-112e-404d-bae2-85d36b83f6f8>

##### Beta - IncomeEstimationModelm

In [ ]:
sq = """ 
select
distinct
r.customerId customer_id ,
r.digitalLoanAccountId,
r.beta_estimated_income,
r.inc_beta_ln_loan_type,
r.inc_beta_ln_education_level,
r.inc_beta_ln_employment_type_new,
r.inc_beta_ln_industry_new,
r.inc_beta_ln_age,
r.inc_beta_ln_brand,
r.inc_beta_ln_city,
r.inc_beta_ln_purpose,
r.inc_beta_ln_osversion_bin,
r.inc_beta_ln_postal_code,
r.inc_beta_ln_gender,
r.inc_beta_ln_doc_type_rolled,
r.inc_beta_ln_cnt_dependents,
r.inc_beta_ln_source_of_funds_new,
r.inc_beta_ln_marital_status_new,
r.inc_beta_encoded_company_name_grouped,
    case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
from `risk_mart.sil_risk_ds_master_20230101_20250309_v2` r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
where r.alpha_estimated_income is not null
;
"""

# data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
data = client.query(sq).result().to_arrow().to_pandas()
print(f"The shape of the dataframe is:\t {data.shape}")

The shape of the dataframe is:	 (349207, 20)


In [ ]:
feature_column = ['inc_beta_ln_loan_type',
       'inc_beta_ln_education_level', 'inc_beta_ln_employment_type_new',
       'inc_beta_ln_industry_new', 'inc_beta_ln_age', 'inc_beta_ln_brand',
       'inc_beta_ln_city', 'inc_beta_ln_purpose', 'inc_beta_ln_osversion_bin',
       'inc_beta_ln_postal_code', 'inc_beta_ln_gender',
       'inc_beta_ln_doc_type_rolled', 'inc_beta_ln_cnt_dependents',
       'inc_beta_ln_source_of_funds_new', 'inc_beta_ln_marital_status_new',
       'inc_beta_encoded_company_name_grouped',]

dfd = transform_data(data, feature_column, a='beta_estimated_income', modelDisplayName='Beta - IncomeEstimationModel') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,3100083,9e66b1fa-ddb0-4c0a-b56b-7b503d2aec65,f587739e-14a9-4693-a9b2-4b19a6149306,8978.322515,2025-11-27T04:36:18.451597,2025-11-27T04:36:18.451597,Beta - IncomeEstimationModel,v1,"{""inc_beta_ln_loan_type"": ""SIL-Instore"", ""inc_...",sil_march 25 models,a2af9b7f-dc09-449d-8eff-8b25ff82bafa,2025-11-27T04:36:18.451597,{},,android
1,2685033,7aca4069-5d30-40f3-a5eb-3691543c3b2c,80b37471-abc7-4d29-8b3c-6a9120d8bbab,16804.042340,2025-11-27T04:36:18.451957,2025-11-27T04:36:18.451957,Beta - IncomeEstimationModel,v1,"{""inc_beta_ln_loan_type"": ""SIL-Instore"", ""inc_...",sil_march 25 models,3c11e4dc-3257-4f91-b9c8-7ca63d18c65e,2025-11-27T04:36:18.451957,{},,android
2,2292934,bb111e15-5572-4239-b695-e4c3ee7084d5,996c4394-1ce8-442b-945f-00d365ce3f20,28010.120009,2025-11-27T04:36:18.452216,2025-11-27T04:36:18.452216,Beta - IncomeEstimationModel,v1,"{""inc_beta_ln_loan_type"": ""SIL-Instore"", ""inc_...",sil_march 25 models,44138ee0-a26c-4fc1-a775-ad6ac37c0adc,2025-11-27T04:36:18.452216,{},,android
3,2805114,484cac8f-caa3-48c3-8972-422930628d10,ec80c29e-3b94-42e8-b08b-855dce2715d1,24999.998695,2025-11-27T04:36:18.452463,2025-11-27T04:36:18.452463,Beta - IncomeEstimationModel,v1,"{""inc_beta_ln_loan_type"": ""SIL-Instore"", ""inc_...",sil_march 25 models,578855df-a542-42d6-aba7-130e498c73e0,2025-11-27T04:36:18.452463,{},,android
4,3089641,307e27d7-9201-4f38-a5e0-0e5ca27cd43d,2c057487-e82e-4930-8f73-20947be5b110,20111.138917,2025-11-27T04:36:18.452722,2025-11-27T04:36:18.452722,Beta - IncomeEstimationModel,v1,"{""inc_beta_ln_loan_type"": ""SIL-Instore"", ""inc_...",sil_march 25 models,3975dfb8-4ee4-4043-a6a3-22e940a06a57,2025-11-27T04:36:18.452722,{},,android


In [ ]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=0e49e2d3-7b77-4638-a22b-711880261d6e>

## Cash

##### Alpha Cash Stack Model

###### Trench 1

In [ ]:
sq = """ 
select 
  r.customer_id,
  r.digitalLoanAccountId, 
  r.stack_score,
  r.demo_score,
  r.apps_score,
  r.credo_score,
  r.cic_score,
  r.stack_score_norm,
  r.ln_os_type osType,
from worktable_data_analysis.cash_alpha_trench1_applied_loans_backscored_20241001_20250930 r;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 58277b53-218b-4e55-9227-f868e76df8b8 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (67807, 9)


In [ ]:
feature_column = ['demo_score', 'apps_score', 'credo_score', 'cic_score', 'stack_score_norm']

dfd = transform_data(data, feature_column, a='stack_score', modelDisplayName='Alpha-Cash-Stack-Model', tc='Trench 1', subscription_name = 'Cash September 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,3539915,04faa4fc-40df-4c35-8fd4-7873f4f27d7a,9cff2b26-a468-40ad-9cb7-ca49cc543329,0.669111,2025-11-27T04:37:58.790717,2025-11-27T04:37:58.790717,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.5744555950383494, ""credo_scor...",Cash September 25 Models,41684675-59a2-4c98-a4dc-2d727fe2a4d6,2025-11-27T04:37:58.790717,{},Trench 1,iOS
1,3556922,e30aad68-3baa-4439-a5cc-8a742648b200,4d7a9ff6-4381-4073-bd11-4ad3e37d48b2,0.316267,2025-11-27T04:37:58.790966,2025-11-27T04:37:58.790966,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.44932706141455475, ""credo_sco...",Cash September 25 Models,64d4fc8b-307f-494f-854f-d16c2927f3cb,2025-11-27T04:37:58.790966,{},Trench 1,iOS
2,3242405,16e5ac4b-e966-4212-9f7b-9d24bb15a8dc,73d4947d-f16c-48db-b046-bd804da2ee97,0.311066,2025-11-27T04:37:58.791119,2025-11-27T04:37:58.791119,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.40506523274601347, ""credo_sco...",Cash September 25 Models,94a6a11d-b687-49b5-b88c-e686686b87e2,2025-11-27T04:37:58.791119,{},Trench 1,iOS
3,3646632,30b64b4a-5982-4e8c-b96e-e064aa605845,eef9167e-4055-4348-b22f-cbf0ac2ddff8,0.533312,2025-11-27T04:37:58.791262,2025-11-27T04:37:58.791262,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.5261059729214972, ""credo_scor...",Cash September 25 Models,8cda6e50-6ec4-4996-8a5f-09395b1553cb,2025-11-27T04:37:58.791262,{},Trench 1,iOS
4,3454325,c96d5134-cf15-4393-a867-2220c7f176cc,90d41d99-ed61-4828-b8f9-a555c224e9f5,0.433226,2025-11-27T04:37:58.791417,2025-11-27T04:37:58.791417,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.39591531683005676, ""credo_sco...",Cash September 25 Models,6b67d3c3-e60a-41dd-b898-602c516ba901,2025-11-27T04:37:58.791417,{},Trench 1,iOS


In [ ]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=89312537-ad92-4f30-a2fd-7d465d68decc>

###### Trench 2

In [ ]:
sq = """ 
select 
  r.customer_id,
  r.digitalLoanAccountId, 
  r.stack_score,
  r.demo_score,
  r.apps_score,
  r.credo_score,
  r.cic_score,
  r.stack_score_norm,
  r.ln_os_type osType,
from worktable_data_analysis.cash_alpha_trench2_applied_loans_backscored_20241001_20250930 r;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 6ccb503b-5920-458b-ad95-d6ff3b7dc241 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (42982, 9)


In [ ]:
feature_column = ['demo_score', 'apps_score', 'credo_score', 'cic_score', 'stack_score_norm']

dfd = transform_data(data, feature_column, a='stack_score', modelDisplayName='Alpha-Cash-Stack-Model', tc='Trench 2', subscription_name = 'Cash September 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,3239336,9386c275-dbf0-4c0a-8d02-8071f592b37d,6435ea9d-969b-4578-b131-61956662d715,0.672123,2025-11-27T04:38:17.112963,2025-11-27T04:38:17.112963,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.6247828133286182, ""credo_scor...",Cash September 25 Models,f7bc1068-1fff-466e-8d06-2e0d092d9fe8,2025-11-27T04:38:17.112963,{},Trench 2,iOS
1,2234712,d5b9351b-4d02-4052-8296-8248cce581fe,ae79e588-2820-4544-b048-fae6d1c37df9,0.618780,2025-11-27T04:38:17.113261,2025-11-27T04:38:17.113261,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.4714931796785411, ""credo_scor...",Cash September 25 Models,1d3e72a5-55e0-4bbb-ad0a-abda8899b4b0,2025-11-27T04:38:17.113261,{},Trench 2,iOS
2,1147827,d5fd2224-f704-414f-8294-4fc98451fa61,fae4fc01-56b8-4628-b168-edd26173075d,0.513876,2025-11-27T04:38:17.113435,2025-11-27T04:38:17.113435,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.5756542624288734, ""credo_scor...",Cash September 25 Models,7dc8f17b-e307-4c1d-bbf8-1cafff0e806e,2025-11-27T04:38:17.113435,{},Trench 2,iOS
3,1924318,db06a8d7-5eaa-42f0-8bca-c2709a7bfe39,e515c17a-6733-4ad3-a575-55960b8fb300,0.687939,2025-11-27T04:38:17.113590,2025-11-27T04:38:17.113590,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.575024075027459, ""credo_score...",Cash September 25 Models,432408d4-87e6-475f-b1ea-11937c95313d,2025-11-27T04:38:17.113590,{},Trench 2,iOS
4,3042999,517586f3-eb4c-4401-9ccf-f12da8d552fc,87cb44ca-c7ab-474d-aec2-2ffc94609505,0.585185,2025-11-27T04:38:17.113758,2025-11-27T04:38:17.113758,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.646694460802266, ""credo_score...",Cash September 25 Models,973fb18a-ecdc-494b-a340-807c3f98921a,2025-11-27T04:38:17.113758,{},Trench 2,iOS


In [ ]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=735bc950-2f5f-4df2-b2ee-5d39c471d215>

###### Trench 3

In [ ]:
sq = """ 
select 
  r.customer_id,
  r.digitalLoanAccountId, 
  r.stack_score,
  r.demo_score,
  r.apps_score,
  r.credo_score,
  r.cic_score,
  r.stack_score_norm,
  r.ln_os_type osType,
from worktable_data_analysis.cash_alpha_trench3_applied_loans_backscored_20241001_20250930 r;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID e6f63fb8-51e4-45e3-92bc-8f4a3a0b8552 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (18269, 9)


In [ ]:
feature_column = ['demo_score', 'apps_score', 'credo_score', 'cic_score', 'stack_score_norm']

dfd = transform_data(data, feature_column, a='stack_score', modelDisplayName='Alpha-Cash-Stack-Model', tc='Trench 3', subscription_name = 'Cash September 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,3169262,96426bd5-17d6-4fe0-bd14-3d7aa84dbe34,591ece13-7fc9-4d3e-8956-6b5b886c5cb3,0.523043,2025-11-27T04:38:30.214279,2025-11-27T04:38:30.214279,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.424909388603803, ""credo_score...",Cash September 25 Models,58106c0d-0af3-4338-bfd9-0ff4b537c298,2025-11-27T04:38:30.214279,{},Trench 3,iOS
1,1494932,4c192f35-a31e-438e-8fd0-ec7273a5ae39,3906fc9a-06a3-4d88-a8ff-d57d2a3ce1d7,0.415618,2025-11-27T04:38:30.214569,2025-11-27T04:38:30.214569,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.3984364213949569, ""credo_scor...",Cash September 25 Models,a3b0211f-534d-47e1-a147-673238d5b436,2025-11-27T04:38:30.214569,{},Trench 3,iOS
2,2735400,fb47df8c-0a1a-45c1-a10e-d2b53b9e2453,a1223c2b-ae46-44e9-af7d-a113dc7d626d,0.373629,2025-11-27T04:38:30.214797,2025-11-27T04:38:30.214797,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.21634670864737435, ""credo_sco...",Cash September 25 Models,81cad433-7821-4829-bd97-d1e5601c704b,2025-11-27T04:38:30.214797,{},Trench 3,iOS
3,1331628,ff48b3ad-9111-4ffe-9a58-4e838afc0410,e5434a64-6780-4939-8003-d019273daf06,0.400812,2025-11-27T04:38:30.215018,2025-11-27T04:38:30.215018,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.49639525992168604, ""credo_sco...",Cash September 25 Models,86e724aa-6c84-4f4c-a722-49db8d7f4391,2025-11-27T04:38:30.215018,{},Trench 3,iOS
4,2225608,485aac6c-1684-4698-be83-c08fdb74a1fd,f25427af-b4c9-4b33-81ab-52b17ee4b674,0.397827,2025-11-27T04:38:30.215231,2025-11-27T04:38:30.215231,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.5367553576956362, ""credo_scor...",Cash September 25 Models,c5db2d2c-c02a-4f98-8d11-35aaaaa75aab,2025-11-27T04:38:30.215231,{},Trench 3,iOS


In [ ]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=d87e679b-7e31-47d0-a303-14e029d6dcdc>

##### Alpha Cash CIC Model

###### Trench 1

In [ ]:
sq = """ 
select 
lmt.customerId customer_id,
r.digitalLoanAccountId,
r.ca_cic_score,
r.max_age_all_contracts_snapshot,
r.ratio_overdue_contracts_to_granted_contracts,
r.ScoreRange,
r.ln_loan_level_user_type,
r.has_ever_been_overdue,
r.latest_granted_contract_overdue_flag,
r.ratio_closed_over_new_granted_cnt_24M,
r.ratio_risky_contracts_to_granted_contracts,
r.Short_and_Term_Loans_granted_contracts_cnt_24M,
r.flg_zero_non_granted_ever,
r.Personal_Loans_granted_contracts_amt_24M,
r.CreditAvgCreditLimit,
r.flg_zero_granted_ever,
    case when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%os%' then 'ios'
    when lower(lmt.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
from worktable_data_analysis.cash_alpha_cic_all_applied_backscored_20240901_20250930 r
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where trench_category = 'Trench 1'
;
"""

data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")


Job ID 50775151-3900-4722-850c-ac0214bcede9 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (77967, 17)


In [ ]:
data.columns

Index(['customer_id', 'digitalLoanAccountId', 'ca_cic_score',
       'max_age_all_contracts_snapshot',
       'ratio_overdue_contracts_to_granted_contracts', 'ScoreRange',
       'ln_loan_level_user_type', 'has_ever_been_overdue',
       'latest_granted_contract_overdue_flag',
       'ratio_closed_over_new_granted_cnt_24M',
       'ratio_risky_contracts_to_granted_contracts',
       'Short_and_Term_Loans_granted_contracts_cnt_24M',
       'flg_zero_non_granted_ever', 'Personal_Loans_granted_contracts_amt_24M',
       'CreditAvgCreditLimit', 'flg_zero_granted_ever', 'osType'],
      dtype='object')

In [ ]:
feature_column = ['max_age_all_contracts_snapshot',
       'ratio_overdue_contracts_to_granted_contracts', 'ScoreRange',
       'ln_loan_level_user_type', 'has_ever_been_overdue',
       'latest_granted_contract_overdue_flag',
       'ratio_closed_over_new_granted_cnt_24M',
       'ratio_risky_contracts_to_granted_contracts',
       'Short_and_Term_Loans_granted_contracts_cnt_24M',
       'flg_zero_non_granted_ever', 'Personal_Loans_granted_contracts_amt_24M',
       'CreditAvgCreditLimit', 'flg_zero_granted_ever', 'ca_cic_score']

dfd = transform_data(data, feature_column, a='ca_cic_score', modelDisplayName='Alpha-Cash-CIC-Model', tc='Trench 1', subscription_name = 'Cash September 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,3308203,62376634-71af-4b67-8c75-40819661b965,b1153bfa-00c1-4f9c-bbdd-9f5044b528cd,0.398100,2025-11-27T04:38:36.817722,2025-11-27T04:38:36.817722,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 2103.0, ""ra...",Cash September 25 Models,ea72e81b-a492-467f-8f04-b4c8e1eba82d,2025-11-27T04:38:36.817722,{},Trench 1,android
1,3227509,f59a7797-d451-4df1-a4cf-1268b907a2b6,8afc9346-4c37-44bb-8e32-ca79c355bdaf,0.490621,2025-11-27T04:38:36.818099,2025-11-27T04:38:36.818099,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 2324.0, ""ra...",Cash September 25 Models,18f60403-7061-4234-959f-cf9e00c9d846,2025-11-27T04:38:36.818099,{},Trench 1,ios
2,3366922,dae741d5-ed10-44de-befa-bc93002b1ade,49bb61b2-91fc-4401-83e3-6c791497d03e,0.582848,2025-11-27T04:38:36.818367,2025-11-27T04:38:36.818367,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 1445.0, ""ra...",Cash September 25 Models,92804952-623e-469f-bd0f-9988e472b959,2025-11-27T04:38:36.818367,{},Trench 1,android
3,3623728,5d45c75a-2f1e-430f-92cc-5b878c754658,d805beef-534f-46d7-8e53-68757c32cda0,0.651699,2025-11-27T04:38:36.818613,2025-11-27T04:38:36.818613,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 2075.0, ""ra...",Cash September 25 Models,2c15ba1e-72db-4e96-9bfd-e0312a58d144,2025-11-27T04:38:36.818613,{},Trench 1,ios
4,3450703,1b4e7ff5-8f4b-4105-8769-f63d29461f7f,d5b14548-6c25-4af2-b6ea-0fee762fd439,0.498188,2025-11-27T04:38:36.818865,2025-11-27T04:38:36.818865,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 2836.0, ""ra...",Cash September 25 Models,d520cda4-1d10-4181-9d14-37301377f6cd,2025-11-27T04:38:36.818865,{},Trench 1,ios


In [ ]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=e581f71b-a751-4b84-8a98-2ca241073c7c>

###### Trench 2

In [ ]:
sq = """ 
select 
lmt.customerId customer_id,
r.digitalLoanAccountId,
r.ca_cic_score,
r.max_age_all_contracts_snapshot,
r.ratio_overdue_contracts_to_granted_contracts,
r.ScoreRange,
r.ln_loan_level_user_type,
r.has_ever_been_overdue,
r.latest_granted_contract_overdue_flag,
r.ratio_closed_over_new_granted_cnt_24M,
r.ratio_risky_contracts_to_granted_contracts,
r.Short_and_Term_Loans_granted_contracts_cnt_24M,
r.flg_zero_non_granted_ever,
r.Personal_Loans_granted_contracts_amt_24M,
r.CreditAvgCreditLimit,
r.flg_zero_granted_ever,
    case when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%os%' then 'ios'
    when lower(lmt.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
from worktable_data_analysis.cash_alpha_cic_all_applied_backscored_20240901_20250930 r
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where trench_category = 'Trench 2'
;
"""


data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 40c6910d-dd79-43cd-8e21-48b3f7eaa351 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (48479, 17)


In [ ]:
feature_column = ['max_age_all_contracts_snapshot',
       'ratio_overdue_contracts_to_granted_contracts', 'ScoreRange',
       'ln_loan_level_user_type', 'has_ever_been_overdue',
       'latest_granted_contract_overdue_flag',
       'ratio_closed_over_new_granted_cnt_24M',
       'ratio_risky_contracts_to_granted_contracts',
       'Short_and_Term_Loans_granted_contracts_cnt_24M',
       'flg_zero_non_granted_ever', 'Personal_Loans_granted_contracts_amt_24M',
       'CreditAvgCreditLimit', 'flg_zero_granted_ever', 'ca_cic_score']

dfd = transform_data(data, feature_column, a='ca_cic_score', modelDisplayName='Alpha-Cash-CIC-Model', tc='Trench 2', subscription_name = 'Cash September 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,2283551,e06d12e7-5c14-4f52-80fc-3c69130e2fd7,b155a7cc-997e-4385-a20e-50f65dd3c69d,0.562817,2025-11-27T04:39:01.444908,2025-11-27T04:39:01.444908,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 2348.0, ""ra...",Cash September 25 Models,d5470bf2-18eb-4d5f-9b65-571824f04539,2025-11-27T04:39:01.444908,{},Trench 2,ios
1,3469091,e9c6c6b9-2446-4bfa-a244-f2269b214c1e,54a13cf9-78f6-4a68-9b62-408c4f57d5c7,0.672037,2025-11-27T04:39:01.445278,2025-11-27T04:39:01.445278,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 405.0, ""rat...",Cash September 25 Models,985e06e4-a317-43c7-912a-40538d8db42c,2025-11-27T04:39:01.445278,{},Trench 2,android
2,2482438,0dce1335-7305-455f-bd77-60a54c66c745,7f59009a-328b-483c-8219-63b18295d7ef,0.391288,2025-11-27T04:39:01.445546,2025-11-27T04:39:01.445546,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 3470.0, ""ra...",Cash September 25 Models,25fd54f3-6db8-4416-b4a9-2de87a8c1a76,2025-11-27T04:39:01.445546,{},Trench 2,ios
3,3300539,02636fad-f0ef-4cc8-94c0-f19054e0d25e,bebe33a0-befe-4f8b-ba43-c394e471b9ee,0.455675,2025-11-27T04:39:01.445777,2025-11-27T04:39:01.445777,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 3093.0, ""ra...",Cash September 25 Models,f415b8fc-c09e-47bc-8493-65cd2ce5cb65,2025-11-27T04:39:01.445777,{},Trench 2,ios
4,2109520,b7b2c866-f31d-48a7-aac4-374d91c2032c,d2ab56fe-e126-442c-811e-91f2991e76e8,0.639888,2025-11-27T04:39:01.446042,2025-11-27T04:39:01.446042,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 804.0, ""rat...",Cash September 25 Models,72b3fa41-e2bd-44a6-8e96-b70e639b47fd,2025-11-27T04:39:01.446042,{},Trench 2,android


In [ ]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=2daed865-bc9a-45eb-9559-65c3854c197f>

###### Trench 3

In [ ]:
sq = """ 
select 
lmt.customerId customer_id,
r.digitalLoanAccountId,
r.ca_cic_score,
r.max_age_all_contracts_snapshot,
r.ratio_overdue_contracts_to_granted_contracts,
r.ScoreRange,
r.ln_loan_level_user_type,
r.has_ever_been_overdue,
r.latest_granted_contract_overdue_flag,
r.ratio_closed_over_new_granted_cnt_24M,
r.ratio_risky_contracts_to_granted_contracts,
r.Short_and_Term_Loans_granted_contracts_cnt_24M,
r.flg_zero_non_granted_ever,
r.Personal_Loans_granted_contracts_amt_24M,
r.CreditAvgCreditLimit,
r.flg_zero_granted_ever,
    case when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%os%' then 'ios'
    when lower(lmt.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
from worktable_data_analysis.cash_alpha_cic_all_applied_backscored_20240901_20250930 r
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where trench_category = 'Trench 3'
;
"""


data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID bece4a40-6303-4c57-a95c-0eecae7c383a successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (19300, 17)


In [ ]:
feature_column = ['max_age_all_contracts_snapshot',
       'ratio_overdue_contracts_to_granted_contracts', 'ScoreRange',
       'ln_loan_level_user_type', 'has_ever_been_overdue',
       'latest_granted_contract_overdue_flag',
       'ratio_closed_over_new_granted_cnt_24M',
       'ratio_risky_contracts_to_granted_contracts',
       'Short_and_Term_Loans_granted_contracts_cnt_24M',
       'flg_zero_non_granted_ever', 'Personal_Loans_granted_contracts_amt_24M',
       'CreditAvgCreditLimit', 'flg_zero_granted_ever', 'ca_cic_score']

dfd = transform_data(data, feature_column, a='ca_cic_score', modelDisplayName='Alpha-Cash-CIC-Model', tc='Trench 3', subscription_name = 'Cash September 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,2013227,7df81a17-a196-46cc-9394-afb6f565d08c,e83e8665-ab1e-4f64-9b6b-1d50d740b69a,0.510133,2025-11-27T04:39:18.230475,2025-11-27T04:39:18.230475,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 912.0, ""rat...",Cash September 25 Models,c0dfd370-9f0a-47c5-9b38-aef66f172f7c,2025-11-27T04:39:18.230475,{},Trench 3,android
1,2020120,ac10377e-9d1a-4f83-892b-822d50035111,13eb4a3e-141b-4c97-936a-4a874501512c,0.371622,2025-11-27T04:39:18.230867,2025-11-27T04:39:18.230867,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 2834.0, ""ra...",Cash September 25 Models,0b9b59f6-112f-45ec-893c-ea3b0c7126db,2025-11-27T04:39:18.230867,{},Trench 3,android
2,1506483,77b3630c-541a-45a5-b289-7e3292af795d,6078c518-a10e-44cc-bd28-aca99c217f54,0.256549,2025-11-27T04:39:18.231129,2025-11-27T04:39:18.231129,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 5627.0, ""ra...",Cash September 25 Models,ed9cea09-11f6-462e-8b37-2311858d61f4,2025-11-27T04:39:18.231129,{},Trench 3,android
3,1934097,dc56f700-2f45-4c76-a17b-67ef3200d7d7,dcbedd60-4e2a-4f9e-b10d-ccb5a53e9d85,0.510801,2025-11-27T04:39:18.231380,2025-11-27T04:39:18.231380,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 1052.0, ""ra...",Cash September 25 Models,c2f713a6-1cc6-4b8e-bdd1-de7157210274,2025-11-27T04:39:18.231380,{},Trench 3,android
4,2751110,0b21474c-ed21-4ed0-a7de-06188721b6fb,63ee794c-3b0f-45a6-b153-166b4a7e4bea,0.504298,2025-11-27T04:39:18.231614,2025-11-27T04:39:18.231614,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 1996.0, ""ra...",Cash September 25 Models,4f9f8c9f-b4a7-4093-a9a1-db7086ea677b,2025-11-27T04:39:18.231614,{},Trench 3,android


In [ ]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=f7e7e145-d2a4-48a1-ba09-2291bc5abe41>

##### Beta-Cash-Demo-Model

###### Trench 1

In [ ]:
sq = """ 
select 
r.customerId customer_id,
r.digitalLoanAccountId,
r.c_demo_score Beta_Cash_Demo_Score,
r.ln_vas_opted_flag, 
r.ln_self_dec_income, 
r.ln_age,
r.ln_source_funds_new_bin, 
r.ln_loan_level_user_type,
r.ln_industry_new_cat_bin, 
r.ln_marital_status,
r.ln_doc_type_rolled, 
r.ln_education_level,
r.ln_ref2_type, 
r.ln_email_primary_domain, 
r.ln_province_bin,
    case when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%os%' then 'ios'
    when lower(lmt.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
from worktable_data_analysis.cash_beta_demo_all_applied_backscored_20241001_20250930 r
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where trench_category = 'Trench 1'
;

"""

data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID af3007ee-3aec-4638-974a-99c090214678 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (324441, 16)


In [ ]:
feature_column = ['ln_vas_opted_flag',
       'ln_self_dec_income', 'ln_age',
       'ln_source_funds_new_bin', 'ln_loan_level_user_type',
       'ln_industry_new_cat_bin',
       'ln_marital_status',
       'ln_doc_type_rolled',
       'ln_education_level',
       'ln_ref2_type', 'ln_email_primary_domain',
       'ln_province_bin','Beta_Cash_Demo_Score']

dfd = transform_data(data, feature_column, a='Beta_Cash_Demo_Score', modelDisplayName='Beta-Cash-Demo-Model', tc='Trench 1', subscription_name = 'Cash September 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,3084733,717d5c32-4bf5-44c7-bf5e-b1b1a79c8f80,dd4b3ae8-47db-467b-a79c-42a892e37403,0.437895,2025-11-27T04:39:30.251361,2025-11-27T04:39:30.251361,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""0"", ""ln_self_dec_income...",Cash September 25 Models,27caaad1-c4c9-431c-b23c-5cb876482b2a,2025-11-27T04:39:30.251361,{},Trench 1,ios
1,3278353,37031cc6-d1e7-41e8-a8ef-8a35f9be69b8,7d131813-14fd-4846-ba50-def3f6ca3eb1,0.359117,2025-11-27T04:39:30.251796,2025-11-27T04:39:30.251796,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""0"", ""ln_self_dec_income...",Cash September 25 Models,a04f58d8-a2f5-4a8f-8a5a-77b104e8421e,2025-11-27T04:39:30.251796,{},Trench 1,android
2,3185027,972574fe-2b06-491e-b818-859db9d08698,af0e92ec-683c-4a49-9f08-5d994a2097cd,0.438755,2025-11-27T04:39:30.252085,2025-11-27T04:39:30.252085,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""1"", ""ln_self_dec_income...",Cash September 25 Models,00b3c539-cb75-4383-842c-0be4e0c6876c,2025-11-27T04:39:30.252085,{},Trench 1,android
3,2923705,eec18a49-ea1f-4ffa-8f93-887030833704,ee000aaa-09b8-44af-a6db-0ee9ddb75cf5,0.582195,2025-11-27T04:39:30.252338,2025-11-27T04:39:30.252338,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""1"", ""ln_self_dec_income...",Cash September 25 Models,57a5c183-a189-4fdf-aafe-5c574aa01ed6,2025-11-27T04:39:30.252338,{},Trench 1,android
4,3094602,9858ef46-e926-42b2-8d5b-0af216b4df65,33312665-f225-4f43-aa77-385c5db1a4e9,0.584055,2025-11-27T04:39:30.252578,2025-11-27T04:39:30.252578,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""1"", ""ln_self_dec_income...",Cash September 25 Models,a7a0c86f-959d-4259-88b7-13c2e2b86278,2025-11-27T04:39:30.252578,{},Trench 1,android


In [ ]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=acbc1efd-25ed-4577-888d-2db1c3e54873>

###### Trench 2

In [ ]:
sq = """ 
select 
r.customerId customer_id,
r.digitalLoanAccountId,
r.c_demo_score Beta_Cash_Demo_Score,
r.ln_vas_opted_flag, 
r.ln_self_dec_income, 
r.ln_age,
r.ln_source_funds_new_bin, 
r.ln_loan_level_user_type,
r.ln_industry_new_cat_bin, 
r.ln_marital_status,
r.ln_doc_type_rolled, 
r.ln_education_level,
r.ln_ref2_type, 
r.ln_email_primary_domain, 
r.ln_province_bin,
    case when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%os%' then 'ios'
    when lower(lmt.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
from worktable_data_analysis.cash_beta_demo_all_applied_backscored_20241001_20250930 r
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where trench_category = 'Trench 2'
;

"""

data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 484a136e-33b2-4c8b-98cf-9ca262f75608 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (124539, 16)


In [ ]:
feature_column = ['ln_vas_opted_flag',
       'ln_self_dec_income', 'ln_age',
       'ln_source_funds_new_bin', 'ln_loan_level_user_type',
       'ln_industry_new_cat_bin',
       'ln_marital_status',
       'ln_doc_type_rolled',
       'ln_education_level',
       'ln_ref2_type', 'ln_email_primary_domain',
       'ln_province_bin','Beta_Cash_Demo_Score']

dfd = transform_data(data, feature_column, a='Beta_Cash_Demo_Score', modelDisplayName='Beta-Cash-Demo-Model', tc='Trench 2', subscription_name = 'Cash September 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,1939977,970fd73c-ab75-4e9f-89c6-c06d8697c003,9a626f71-f14c-48d4-b735-c612562294e5,0.479310,2025-11-27T04:40:57.349562,2025-11-27T04:40:57.349562,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""1"", ""ln_self_dec_income...",Cash September 25 Models,c39884c8-5a39-464f-8bbb-bd5e7fcac176,2025-11-27T04:40:57.349562,{},Trench 2,android
1,3092910,4a3f7b29-dec9-4e9f-9ce5-bb824a902adf,5ba9c866-2561-4a43-a60f-e7fc38af87a3,0.604591,2025-11-27T04:40:57.349895,2025-11-27T04:40:57.349895,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""1"", ""ln_self_dec_income...",Cash September 25 Models,63dfa05a-4413-4aad-9c52-b53e54882d31,2025-11-27T04:40:57.349895,{},Trench 2,android
2,1593341,8f1e3c76-dbc7-45d0-b02e-248162910c8f,241b33eb-8146-4ae1-8811-a31f2bb60055,0.417494,2025-11-27T04:40:57.350157,2025-11-27T04:40:57.350157,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""1"", ""ln_self_dec_income...",Cash September 25 Models,b65ccddb-e082-43e9-ba2a-accd17ba10e9,2025-11-27T04:40:57.350157,{},Trench 2,ios
3,2914009,07f2759b-53c3-4560-82f3-dba81b4743b5,1ade1868-6f0e-4eb2-bbde-a0d68bc889f6,0.553921,2025-11-27T04:40:57.350392,2025-11-27T04:40:57.350392,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""1"", ""ln_self_dec_income...",Cash September 25 Models,cc74b861-4ecc-4a3f-8ff0-0d6c92ab11b8,2025-11-27T04:40:57.350392,{},Trench 2,android
4,2743295,755e3990-5762-4abf-8a98-a73921768da8,16f9f634-0293-44d7-8f22-413a14654e58,0.405290,2025-11-27T04:40:57.350610,2025-11-27T04:40:57.350610,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""0"", ""ln_self_dec_income...",Cash September 25 Models,be293e15-a899-437e-831c-b0d3d06cad4d,2025-11-27T04:40:57.350610,{},Trench 2,android


In [ ]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=e882bc17-2fad-4163-a67b-6fe8b8b90429>

###### Trench 3

In [ ]:
sq = """ 
select 
r.customerId customer_id,
r.digitalLoanAccountId,
r.c_demo_score Beta_Cash_Demo_Score,
r.ln_vas_opted_flag, 
r.ln_self_dec_income, 
r.ln_age,
r.ln_source_funds_new_bin, 
r.ln_loan_level_user_type,
r.ln_industry_new_cat_bin, 
r.ln_marital_status,
r.ln_doc_type_rolled, 
r.ln_education_level,
r.ln_ref2_type, 
r.ln_email_primary_domain, 
r.ln_province_bin,
    case when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%os%' then 'ios'
    when lower(lmt.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
from worktable_data_analysis.cash_beta_demo_all_applied_backscored_20241001_20250930 r
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where trench_category = 'Trench 3'
;

"""

data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 154a2136-3f3f-4461-8319-1100802fd284 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (42441, 16)


In [ ]:
feature_column = ['ln_vas_opted_flag',
       'ln_self_dec_income', 'ln_age',
       'ln_source_funds_new_bin', 'ln_loan_level_user_type',
       'ln_industry_new_cat_bin',
       'ln_marital_status',
       'ln_doc_type_rolled',
       'ln_education_level',
       'ln_ref2_type', 'ln_email_primary_domain',
       'ln_province_bin','Beta_Cash_Demo_Score']

dfd = transform_data(data, feature_column, a='Beta_Cash_Demo_Score', modelDisplayName='Beta-Cash-Demo-Model', tc='Trench 3', subscription_name = 'Cash September 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,2300320,889791d0-0410-4123-be02-8ff3e8bb540c,f5e4a5c1-6da5-445b-a1e5-ae4eda27cbab,0.405936,2025-11-27T04:41:35.901243,2025-11-27T04:41:35.901243,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""1"", ""ln_self_dec_income...",Cash September 25 Models,7d73e17e-72f4-4616-809e-ae2acd3fd754,2025-11-27T04:41:35.901243,{},Trench 3,android
1,3232147,bb0eefc6-044a-48a0-bbe3-f2ac7a7d5f8e,b2689e33-1915-4e88-8a72-01c50503417a,0.191957,2025-11-27T04:41:35.901598,2025-11-27T04:41:35.901598,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""0"", ""ln_self_dec_income...",Cash September 25 Models,17d5284d-2ea9-4443-bb7e-5c491cc25fd6,2025-11-27T04:41:35.901598,{},Trench 3,android
2,2445601,8197fb05-de3c-474b-9905-feb2b6bb5749,3bc202f8-01f0-439d-91b3-19c9cba70e42,0.514768,2025-11-27T04:41:35.901836,2025-11-27T04:41:35.901836,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""1"", ""ln_self_dec_income...",Cash September 25 Models,89a56514-631f-460a-82b5-92e20a0e3b0d,2025-11-27T04:41:35.901836,{},Trench 3,android
3,2097860,ed81244b-b7c1-4f28-9db0-1ad77a397e13,b51b9d5b-fa83-4619-921d-1ceb016f18fa,0.277808,2025-11-27T04:41:35.902112,2025-11-27T04:41:35.902112,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""0"", ""ln_self_dec_income...",Cash September 25 Models,400cff45-de52-4edb-9e5d-d882c437cf99,2025-11-27T04:41:35.902112,{},Trench 3,android
4,3082851,1b1ca6b8-e0fb-4b99-8c7c-6fb498c2eb2b,111eeb02-e14a-45cd-a6f9-2e1f4524d36b,0.364719,2025-11-27T04:41:35.902389,2025-11-27T04:41:35.902389,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""1"", ""ln_self_dec_income...",Cash September 25 Models,db4eeb39-a709-44bb-8ecf-6035619e713f,2025-11-27T04:41:35.902389,{},Trench 3,ios


In [ ]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=32bfdfd4-d8fc-44f3-83ca-257950bfdfe2>

##### Beta-Cash-Stack-Model

In [ ]:
sq = """ 
select r.customer_id,
r.digitalLoanAccountId,
r.demo_score,
r.apps_score,
r.credo_score,
r.stack_score Beta_cash_stack_score,
r.stack_score_norm,
    case when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%os%' then 'ios'
    when lower(lmt.deviceType) like '%andro%' then 'android'
    else 'ios' end osType
 from worktable_data_analysis.cash_beta_trench1_applied_loans_backscored_20241001_20250930 r
 left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 80b929e2-c6ce-40db-a820-e3d0891b433a successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (324344, 8)


In [ ]:
feature_column = ['demo_score',
       'apps_score', 'credo_score',
       'stack_score', 'stack_score_norm']

dfd = transform_data(data, feature_column, a='Beta_cash_stack_score', modelDisplayName='Beta-Cash-Stack-Model', tc='Trench 1', subscription_name = 'Cash September 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,2985144,b472355a-b33f-4a31-b3c6-4ea7259910b3,3afedea3-9bf6-4fcf-bf04-27de32725b8c,0.708075,2025-11-27T04:41:53.350038,2025-11-27T04:41:53.350038,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.4521037308849204, ""apps_score...",Cash September 25 Models,b1b52533-551c-4079-9513-b756c8ac3c26,2025-11-27T04:41:53.350038,{},Trench 1,android
1,3536235,ca8a4b87-4d50-4ff5-810b-f355c486b1a0,8ebf649f-3675-4c2c-bf04-eccd1d1c4663,0.334382,2025-11-27T04:41:53.350366,2025-11-27T04:41:53.350366,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.2588234606154312, ""apps_score...",Cash September 25 Models,32c95351-f55c-4620-948d-e0c6d61b2907,2025-11-27T04:41:53.350366,{},Trench 1,android
2,3085969,733a9e6b-fef5-4203-be9c-bacb16fa13fc,57495204-2224-42f0-9c81-d0dc8595a11f,0.777074,2025-11-27T04:41:53.350532,2025-11-27T04:41:53.350532,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.5540182642207785, ""apps_score...",Cash September 25 Models,7a502cac-618d-4b91-bde9-263495440e19,2025-11-27T04:41:53.350532,{},Trench 1,android
3,3003611,9a09fb3c-0c4a-4b0a-a9e2-a1460afe6437,eb7f7115-7c79-412a-841f-bfc2c075db18,0.327986,2025-11-27T04:41:53.350678,2025-11-27T04:41:53.350678,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.45160336873656354, ""apps_scor...",Cash September 25 Models,122a5808-1eb1-4dc0-8b39-a0fbacc45001,2025-11-27T04:41:53.350678,{},Trench 1,android
4,2933369,df0560b0-f64d-4b34-af87-6dbfd91b43f7,eca9ab13-f389-4f9b-ade1-b41f0a00aeb7,0.024374,2025-11-27T04:41:53.350818,2025-11-27T04:41:53.350818,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.49576237168698817, ""apps_scor...",Cash September 25 Models,2153bcf6-3c66-47df-b255-91692db53e05,2025-11-27T04:41:53.350818,{},Trench 1,android


In [ ]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=160e5371-91cf-440b-b486-3ee64f014b7a>

###### Trench 2

In [ ]:
sq = """ 
select r.customer_id,
r.digitalLoanAccountId,
r.demo_score,
r.apps_score,
r.credo_score,
r.stack_score Beta_cash_stack_score,
r.stack_score_norm,
    case when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%os%' then 'ios'
    when lower(lmt.deviceType) like '%andro%' then 'android'
    else 'ios' end osType
 from worktable_data_analysis.cash_beta_trench2_applied_loans_backscored_20241001_20250930 r
 left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 7bac7cff-073e-4cb9-8825-3d2730dfc138 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (121682, 8)


In [ ]:
feature_column = ['demo_score',
       'apps_score', 'credo_score',
       'stack_score', 'stack_score_norm']

dfd = transform_data(data, feature_column, a='Beta_cash_stack_score', modelDisplayName='Beta-Cash-Stack-Model', tc='Trench 2', subscription_name = 'Cash September 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,3020640,62ca587b-26cf-4315-b8bd-7394336b3c31,5392dab7-dac0-40cf-a3f2-4d95678450e1,0.628072,2025-11-27T04:42:53.651285,2025-11-27T04:42:53.651285,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.49447644732324386, ""apps_scor...",Cash September 25 Models,e3a4b4bf-a10a-4941-8ddb-84cd19b15667,2025-11-27T04:42:53.651285,{},Trench 2,android
1,3124698,3b6f9656-ef72-41ca-b924-d8cf12c1dcbe,b90e3b05-e99a-4942-8a22-f4fa326e8088,0.465860,2025-11-27T04:42:53.651574,2025-11-27T04:42:53.651574,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.5163647390764557, ""apps_score...",Cash September 25 Models,3718fba5-4789-4fcf-b37a-2e5446f6ba2b,2025-11-27T04:42:53.651574,{},Trench 2,android
2,2568670,18cce9e1-a4ab-4f9c-b765-004c827187f8,aa324363-66b1-428e-bdab-539b2abfdae2,0.396730,2025-11-27T04:42:53.651748,2025-11-27T04:42:53.651748,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.4981699264068175, ""apps_score...",Cash September 25 Models,1b3ca617-479c-4a8b-9bbc-74436c4fe0ed,2025-11-27T04:42:53.651748,{},Trench 2,android
3,3089007,63452d4c-2a63-4dd0-b350-c6ed8a6b2767,80626491-da8d-4cb3-8a4b-18ac0d88cf10,0.498779,2025-11-27T04:42:53.651913,2025-11-27T04:42:53.651913,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.47257087895482075, ""apps_scor...",Cash September 25 Models,81c7c0e4-c82f-4b94-bc28-252d255980fd,2025-11-27T04:42:53.651913,{},Trench 2,android
4,2717669,68753109-b074-41de-ba8d-a6bee1536e47,b9c4b7ec-0a5a-4299-ada4-2a4761cba3df,0.649950,2025-11-27T04:42:53.652058,2025-11-27T04:42:53.652058,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.501923531654108, ""apps_score""...",Cash September 25 Models,3676cf20-e69f-4006-9d5e-101a0ed42d30,2025-11-27T04:42:53.652058,{},Trench 2,android


In [ ]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=5d34ba15-5778-4278-b1cd-fc24414ef888>

###### Trench 3

In [ ]:
sq = """ 
select r.customer_id,
r.digitalLoanAccountId,
r.demo_score,
r.apps_score,
r.credo_score,
r.stack_score Beta_cash_stack_score,
r.stack_score_norm,
    case when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%os%' then 'ios'
    when lower(lmt.deviceType) like '%andro%' then 'android'
    else 'ios' end osType
 from worktable_data_analysis.cash_beta_trench3_applied_loans_backscored_20241001_20250930 r
 left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId;

"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")


Job ID 5c0f3281-6375-41ec-87e6-5c97fc25657c successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (42414, 8)


In [ ]:
feature_column = ['demo_score',
       'apps_score', 'credo_score',
       'stack_score', 'stack_score_norm']

dfd = transform_data(data, feature_column, a='Beta_cash_stack_score', modelDisplayName='Beta-Cash-Stack-Model', tc='Trench 3', subscription_name = 'Cash September 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,2514003,4ce8fe99-17fd-48f9-bd9f-c5ac141eb125,62911b0b-1258-4665-a053-ed7c788bffbd,0.458132,2025-11-27T04:43:18.603516,2025-11-27T04:43:18.603516,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.3814313550470237, ""apps_score...",Cash September 25 Models,bdb05b5a-789f-4510-94a6-6b26545029e7,2025-11-27T04:43:18.603516,{},Trench 3,android
1,2403434,c63e36bd-4bfd-4796-9285-6f104a8284a0,826fd7ff-c3fc-4a63-bf7a-98c030ee8341,0.671637,2025-11-27T04:43:18.603780,2025-11-27T04:43:18.603780,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.4426985639773965, ""apps_score...",Cash September 25 Models,59fca64c-e0de-40c5-81c3-f3b2d27f8e4a,2025-11-27T04:43:18.603780,{},Trench 3,android
2,1758384,0d181820-06d6-419c-a5c7-22ecc41727d6,34e1f1c6-3057-47d7-bf8a-2866a6a38a35,0.293517,2025-11-27T04:43:18.603956,2025-11-27T04:43:18.603956,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.41414805499442686, ""apps_scor...",Cash September 25 Models,f588a587-e745-42cc-a188-ebc6611f802d,2025-11-27T04:43:18.603956,{},Trench 3,android
3,3068866,a99d9f85-5e53-43a5-a556-fd51d21ebff4,2d4bf529-cc05-4f2e-9eed-413d75d068cb,0.507325,2025-11-27T04:43:18.604111,2025-11-27T04:43:18.604111,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.36186933291916024, ""apps_scor...",Cash September 25 Models,b19434f5-1edd-40d4-85cb-9a35e8910506,2025-11-27T04:43:18.604111,{},Trench 3,android
4,2569290,985a2a1d-e8aa-469d-aed4-e87af84486a2,d3e3c1b0-2a57-4ff5-b7b6-2515305d3406,0.710462,2025-11-27T04:43:18.604261,2025-11-27T04:43:18.604261,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.40730580387473986, ""apps_scor...",Cash September 25 Models,67be3d69-fc3c-44a0-8604-6dadc32a8b5e,2025-11-27T04:43:18.604261,{},Trench 3,android


In [ ]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=8e3f001a-09e8-4098-9b3b-f1d797e4e7fe>

#####  Beta Cash AppScore Model

###### Trench 1

In [ ]:
## query to add the trench category trench 1 and trench 2

# create or replace table risk_mart.applied_quick_loan_new_applicants_20230101_20250930_app_scored_copy as 
# select * from risk_mart.applied_quick_loan_new_applicants_20230101_20250930_app_scored;



# ALTER TABLE risk_mart.applied_quick_loan_new_applicants_20230101_20250930_app_scored_copy
# ADD COLUMN trench_category STRING;

# MERGE risk_mart.applied_quick_loan_new_applicants_20230101_20250930_app_scored_copy AS target
# USING (
#   SELECT 
#     a.digitalLoanAccountId,
#     CASE 
#       WHEN b.ln_loan_level_user_type = '2_New Applicant'
#            AND DATE_DIFF(DATE(b.ln_appln_submit_datetime), DATE(b.onb_tsa_onboarding_datetime), DAY) <= 30 THEN 'Trench 1'
#       WHEN b.ln_loan_level_user_type = '2_New Applicant'
#            AND DATE_DIFF(DATE(b.ln_appln_submit_datetime), DATE(b.onb_tsa_onboarding_datetime), DAY) > 30 THEN 'Trench 2'
#       WHEN b.ln_loan_level_user_type = '1_Repeat Applicant' THEN 'Trench 3'
#       ELSE NULL
#     END AS trench_category,
#     ROW_NUMBER() OVER (PARTITION BY a.digitalLoanAccountId ORDER BY b.ln_appln_submit_datetime DESC) AS row_num
#   FROM risk_mart.applied_quick_loan_new_applicants_20230101_20250930_app_scored_copy a
#   LEFT JOIN prj-prod-dataplatform.risk_mart.applied_loans_20210701_20250930_trans b
#     ON a.digitalLoanAccountId = b.digitalLoanAccountId
# ) AS source
# ON target.digitalLoanAccountId = source.digitalLoanAccountId
# WHEN MATCHED AND source.row_num = 1 THEN
#   UPDATE SET target.trench_category = source.trench_category;

In [ ]:
sq = """ 
select 
  r.customerId customer_id,
  r.digitalLoanAccountId,
  r.apps_score beta_cash_app_score,  
  r.app_cnt_health_and_fitness_ever app_cnt_health_and_fitness_ever, 
  r.app_cnt_shopping_ever app_cnt_shopping_ever,
  r.app_cnt_crypto_ever app_cnt_crypto_ever, 
  r.app_cnt_driver_ever app_cnt_driver_ever,
  r.app_cnt_payday_180d app_cnt_payday_180d, 
  r.app_cnt_gambling_180d app_cnt_gambling_180d,
  r.app_avg_time_bw_installed_mins_3d app_avg_time_bw_installed_mins_3d,
  r.app_median_time_bw_installed_mins_ever app_median_time_bw_installed_mins_3d,
    case when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%os%' then 'ios'
    when lower(lmt.deviceType) like '%andro%' then 'android'
    else 'ios' end osType
from risk_mart.applied_quick_loan_new_applicants_20230101_20250930_app_scored_copy r 
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where r.trench_category = 'Trench 1';
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 09734c1c-a8da-442b-be7d-9dd0dfa75898 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (386694, 12)


In [ ]:
data.columns

Index(['customer_id', 'digitalLoanAccountId', 'beta_cash_app_score',
       'app_cnt_health_and_fitness_ever', 'app_cnt_shopping_ever',
       'app_cnt_crypto_ever', 'app_cnt_driver_ever', 'app_cnt_payday_180d',
       'app_cnt_gambling_180d', 'app_avg_time_bw_installed_mins_3d',
       'app_median_time_bw_installed_mins_3d', 'osType'],
      dtype='object')

In [ ]:
feature_column = ['beta_cash_app_score',
       'app_cnt_health_and_fitness_ever', 'app_cnt_shopping_ever',
       'app_cnt_crypto_ever', 'app_cnt_driver_ever', 'app_cnt_payday_180d',
       'app_cnt_gambling_180d', 'app_avg_time_bw_installed_mins_3d',
       'app_median_time_bw_installed_mins_3d']

dfd = transform_data(data, feature_column, a='beta_cash_app_score', modelDisplayName='Beta-Cash-AppScore-Model', tc='Trench 1', subscription_name = 'Cash September 25 Models') 
dfd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386694 entries, 0 to 386693
Data columns (total 15 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   customerId            386694 non-null  int64  
 1   digitalLoanAccountId  386694 non-null  object 
 2   crifApplicationId     386694 non-null  object 
 3   prediction            386694 non-null  float64
 4   start_time            386694 non-null  object 
 5   end_time              386694 non-null  object 
 6   modelDisplayName      386694 non-null  object 
 7   modelVersionId        386694 non-null  object 
 8   calcFeature           386694 non-null  object 
 9   subscription_name     386694 non-null  object 
 10  message_id            386694 non-null  object 
 11  publish_time          386694 non-null  object 
 12  attributes            386694 non-null  object 
 13  trenchCategory        386694 non-null  object 
 14  deviceOs              386694 non-null  object 
dtype

In [ ]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=f97d0c6d-4977-49c4-b115-24c48571e0a1>

###### Trench 2

In [ ]:
sq = """ 
select 
  r.customerId customer_id,
  r.digitalLoanAccountId,
  r.apps_score beta_cash_app_score,  
  r.app_cnt_health_and_fitness_ever app_cnt_health_and_fitness_ever, 
  r.app_cnt_shopping_ever app_cnt_shopping_ever,
  r.app_cnt_crypto_ever app_cnt_crypto_ever, 
  r.app_cnt_driver_ever app_cnt_driver_ever,
  r.app_cnt_payday_180d app_cnt_payday_180d, 
  r.app_cnt_gambling_180d app_cnt_gambling_180d,
  r.app_avg_time_bw_installed_mins_3d app_avg_time_bw_installed_mins_3d,
  r.app_median_time_bw_installed_mins_ever app_median_time_bw_installed_mins_3d,
    case when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%os%' then 'ios'
    when lower(lmt.deviceType) like '%andro%' then 'android'
    else 'ios' end osType
from risk_mart.applied_quick_loan_new_applicants_20230101_20250930_app_scored_copy r 
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where r.trench_category = 'Trench 2';
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID ada59552-39b3-440d-aec8-d2c2132400ba successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (122895, 12)


In [ ]:
feature_column = ['beta_cash_app_score',
       'app_cnt_health_and_fitness_ever', 'app_cnt_shopping_ever',
       'app_cnt_crypto_ever', 'app_cnt_driver_ever', 'app_cnt_payday_180d',
       'app_cnt_gambling_180d', 'app_avg_time_bw_installed_mins_3d',
       'app_median_time_bw_installed_mins_3d']

dfd = transform_data(data, feature_column, a='beta_cash_app_score', modelDisplayName='Beta-Cash-AppScore-Model', tc='Trench 2', subscription_name = 'Cash September 25 Models') 
dfd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122895 entries, 0 to 122894
Data columns (total 15 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   customerId            122895 non-null  int64  
 1   digitalLoanAccountId  122895 non-null  object 
 2   crifApplicationId     122895 non-null  object 
 3   prediction            122895 non-null  float64
 4   start_time            122895 non-null  object 
 5   end_time              122895 non-null  object 
 6   modelDisplayName      122895 non-null  object 
 7   modelVersionId        122895 non-null  object 
 8   calcFeature           122895 non-null  object 
 9   subscription_name     122895 non-null  object 
 10  message_id            122895 non-null  object 
 11  publish_time          122895 non-null  object 
 12  attributes            122895 non-null  object 
 13  trenchCategory        122895 non-null  object 
 14  deviceOs              122895 non-null  object 
dtype

In [ ]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=54beb7b5-a72c-460d-b722-ade2eede5250>

###### Trench 3

In [ ]:
sq = """   
 select 
  r.customerId customer_id, 
  r.digitalLoanAccountId,
  r.apps_score beta_cash_app_score,
  r.app_cnt_health_and_fitness_ever_binned app_cnt_health_and_fitness_ever,
  r.app_cnt_productivity_ever_binned app_cnt_productivity_ever, 
  r.app_cnt_rated_for_18plus_ever_binned app_cnt_rated_for_18plus_ever,
  r.app_cnt_books_and_reference_ever_binned app_cnt_books_and_reference_ever, 
  r.app_cnt_gaming_180d_binned app_cnt_gaming_180d,
  r.app_cnt_absence_tag_365d_binned app_cnt_absence_tag_365d,
  r.app_last_payday_install_to_apply_days_binned app_last_payday_install_to_apply_days,
  r.app_cnt_absence_tag_365d_binned,
  r.app_cnt_gaming_180d_binned,
  r.app_cnt_productivity_ever_binned,
  r.app_cnt_rated_for_18plus_ever_binned,
  r.app_cnt_health_and_fitness_ever_binned,
  r.app_cnt_books_and_reference_ever_binned,
  r.app_last_payday_install_to_apply_days_binned,
  r.ln_user_type,
  case when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%os%' then 'ios'
    when lower(lmt.deviceType) like '%andro%' then 'android'
    else 'ios' end osType
from risk_mart.b_score_model_applied_loans_cash_20240101_20250930_app_scored r
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId

"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 5bab3b2e-f1e0-44e8-a022-461846f7dd67 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (67403, 19)


In [ ]:
feature_column = ['beta_cash_app_score',
       'app_cnt_health_and_fitness_ever', 'app_cnt_productivity_ever',
       'app_cnt_rated_for_18plus_ever', 'app_cnt_books_and_reference_ever',
       'app_cnt_gaming_180d', 'app_cnt_absence_tag_365d',
       'app_last_payday_install_to_apply_days',
       'app_cnt_absence_tag_365d_binned', 'app_cnt_gaming_180d_binned',
       'app_cnt_productivity_ever_binned',
       'app_cnt_rated_for_18plus_ever_binned',
       'app_cnt_health_and_fitness_ever_binned',
       'app_cnt_books_and_reference_ever_binned',
       'app_last_payday_install_to_apply_days_binned', 'ln_user_type']

dfd = transform_data(data, feature_column, a='beta_cash_app_score', modelDisplayName='Beta-Cash-AppScore-Model', tc='Trench 3', subscription_name = 'Cash September 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,1534579,587d316b-afc0-4d7b-9b54-2ee247a679dd,2564130a-0e7f-43b0-b014-56a918fe73d6,0.474986,2025-11-27T04:45:33.274239,2025-11-27T04:45:33.274239,Beta-Cash-AppScore-Model,v1,"{""beta_cash_app_score"": 0.4749856955909727, ""a...",Cash September 25 Models,6870bfa2-8181-4fe5-b3c3-9c57e81bbb08,2025-11-27T04:45:33.274239,{},Trench 3,android
1,2821952,08abc185-e7b5-45c4-92e5-722c46ec0fb2,b7be0340-ce1e-449b-adcc-91f5b36fdf5b,0.493479,2025-11-27T04:45:33.274621,2025-11-27T04:45:33.274621,Beta-Cash-AppScore-Model,v1,"{""beta_cash_app_score"": 0.49347855191279816, ""...",Cash September 25 Models,4d315a33-6c8e-4d16-9dfa-75b4f0e9fc68,2025-11-27T04:45:33.274621,{},Trench 3,android
2,2970058,d58a876e-c410-4800-81e3-926edd289fc9,3f54f6fd-6020-44dd-86f0-776c5671dd16,0.471427,2025-11-27T04:45:33.274909,2025-11-27T04:45:33.274909,Beta-Cash-AppScore-Model,v1,"{""beta_cash_app_score"": 0.47142676478563356, ""...",Cash September 25 Models,c0b5f4f8-0d15-47b2-a8e3-0876f099f1ad,2025-11-27T04:45:33.274909,{},Trench 3,android
3,2156280,1e356a9d-1bfe-4376-a44e-43f880239935,3ee984d5-27ee-4dc2-af73-d5658a20f00b,0.515776,2025-11-27T04:45:33.275212,2025-11-27T04:45:33.275212,Beta-Cash-AppScore-Model,v1,"{""beta_cash_app_score"": 0.5157757106487761, ""a...",Cash September 25 Models,0c6dc017-2bef-41c5-b670-af16003c821a,2025-11-27T04:45:33.275212,{},Trench 3,android
4,2121189,dc6b3bef-60c3-4fe0-a3b0-5c9654647708,544adc4f-3013-4c52-9836-62513a1fb828,0.489441,2025-11-27T04:45:33.275487,2025-11-27T04:45:33.275487,Beta-Cash-AppScore-Model,v1,"{""beta_cash_app_score"": 0.48944090843843197, ""...",Cash September 25 Models,eb04674f-fa91-4be7-8dc3-8a871ae6841a,2025-11-27T04:45:33.275487,{},Trench 3,android


In [ ]:
dfd.info()
dfd['customerId'] = pd.to_numeric(dfd['customerId'], errors='coerce')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67403 entries, 0 to 67402
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   customerId            67403 non-null  object 
 1   digitalLoanAccountId  67403 non-null  object 
 2   crifApplicationId     67403 non-null  object 
 3   prediction            67403 non-null  float64
 4   start_time            67403 non-null  object 
 5   end_time              67403 non-null  object 
 6   modelDisplayName      67403 non-null  object 
 7   modelVersionId        67403 non-null  object 
 8   calcFeature           67403 non-null  object 
 9   subscription_name     67403 non-null  object 
 10  message_id            67403 non-null  object 
 11  publish_time          67403 non-null  object 
 12  attributes            67403 non-null  object 
 13  trenchCategory        67403 non-null  object 
 14  deviceOs              67403 non-null  object 
dtypes: float64(1), obje

In [ ]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=96b6d484-7484-4706-9265-f023f0b2408a>

# End